# irp-dbk24 - "Optimising Demand Response Strategies for Carbon-Intelligent Electricity Use"

# Developing Marginal Emissions Models

**NOTEBOOK PURPOSE(S):**

**LIMITATIONS:**

**NOTEBOOK OUTPUTS:**

    

### Importing Libraries

In [1]:
# ────────────────────────────────────────────────────────────────────────────
# Future (must be first)
# ────────────────────────────────────────────────────────────────────────────
from __future__ import annotations

# ────────────────────────────────────────────────────────────────────────────
# Jupyter/Notebook Setup
# ────────────────────────────────────────────────────────────────────────────
%matplotlib inline
from IPython.display import display

# ────────────────────────────────────────────────────────────────────────────
# Standard Library
# ────────────────────────────────────────────────────────────────────────────
import binascii
import calendar
import hashlib
import inspect
import json
import logging
import math
import os
import random
import re
import time
from contextlib import contextmanager
from copy import deepcopy
from dataclasses import dataclass
from datetime import datetime, timedelta
from functools import partial, wraps
from itertools import combinations, product
from multiprocessing import Lock, Manager, Pool, cpu_count
from multiprocessing.pool import ThreadPool
from pathlib import Path
from typing import (
    Any, Callable, Dict, Iterable, List, Mapping, Optional,
    Sequence, Tuple, Union
)
from zoneinfo import ZoneInfo

# ────────────────────────────────────────────────────────────────────────────
# Core Data Handling
# ────────────────────────────────────────────────────────────────────────────
import numpy as np
import pandas as pd
import polars as pl

from functools import reduce
from operator import add

# ────────────────────────────────────────────────────────────────────────────
# Machine Learning & Statistics
# ────────────────────────────────────────────────────────────────────────────
from feature_engine.creation import CyclicalFeatures
from pygam import LinearGAM, l, s
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import kurtosis, skew, zscore

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    root_mean_squared_error,
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, SplineTransformer
from sklearn.utils.validation import check_is_fitted

# ────────────────────────────────────────────────────────────────────────────
# Visualization
# ────────────────────────────────────────────────────────────────────────────
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import seaborn as sns

# ────────────────────────────────────────────────────────────────────────────
# Geospatial
# ────────────────────────────────────────────────────────────────────────────
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.wkb import loads
from pyproj import Proj, transform

## Functions

### Utilities

#### CSV File Handling

In [2]:
def _drop_hash_from_part(
        part_path: Path,
        model_hash: str,
        *,
        chunk_size: int = 200_000,
        delete_if_empty: bool = False,
) -> int:
    """
    Remove rows with model_id_hash == `model_hash` from a CSV part file.

    - Streams in chunks (no huge memory spikes)
    - Writes to a temp file, then atomically replaces the original
    - Returns number of rows dropped
    - If all rows are dropped:
        • delete the file if `delete_if_empty=True`
        • otherwise keep a header-only CSV

    Parameters
    ----------
    part_path : Path
        CSV file to edit in place.
    model_hash : str
        Value to filter out from the 'model_id_hash' column.
    chunk_size : int, default 200_000
        Pandas read_csv chunk size.
    delete_if_empty : bool, default False
        If True and all rows are removed, delete the part file.

    Returns
    -------
    int
        Number of rows removed.
    """
    part_path = Path(part_path)
    if not part_path.exists():
        return 0

    # Quick header check
    try:
        header_df = pd.read_csv(part_path, nrows=0)
    except Exception:
        # Broken file — leave as-is
        return 0
    if "model_id_hash" not in header_df.columns:
        return 0

    dropped = 0
    kept = 0
    tmp_path = part_path.with_suffix(part_path.suffix + ".tmp")

    # Ensure no stale tmp
    if tmp_path.exists():
        try:
            tmp_path.unlink()
        except Exception:
            pass

    first_write = True
    try:
        for chunk in pd.read_csv(
            part_path,
            chunksize=chunk_size,
            dtype={"model_id_hash": "string"},  # force string, avoid numeric coercion
        ):
            if "model_id_hash" not in chunk.columns:
                # schema changed mid-file? abort safely
                dropped = 0
                kept = -1
                break
            mask = chunk["model_id_hash"] != model_hash
            kept_chunk = chunk.loc[mask]
            n_dropped = int((~mask).sum())
            dropped += n_dropped
            kept += int(mask.sum())

            if kept_chunk.empty:
                continue

            kept_chunk.to_csv(
                tmp_path,
                index=False,
                mode="w" if first_write else "a",
                header=first_write,
            )
            first_write = False

        # Nothing matched → no change
        if dropped == 0:
            if tmp_path.exists():
                # wrote identical content; discard temp
                try: tmp_path.unlink()
                except Exception: pass
            return 0

        # All rows removed
        if kept == 0:
            if delete_if_empty:
                # Delete original; remove temp if created
                try: part_path.unlink()
                except Exception: pass
                if tmp_path.exists():
                    try: tmp_path.unlink()
                    except Exception: pass
            else:
                # Replace with header-only CSV
                header_df.to_csv(tmp_path, index=False)
                os.replace(tmp_path, part_path)
            return dropped

        # Normal case: replace atomically
        os.replace(tmp_path, part_path)
        return dropped

    finally:
        # Best-effort cleanup
        if tmp_path.exists():
            try: os.remove(tmp_path)
            except Exception: pass


In [3]:
def is_model_logged_rotating_csv(
        model_hash: str,
        base_dir: str | Path,
        file_prefix: str
) -> bool:
    """
    Return True if `model_hash` appears in the rolling-log index for `file_prefix`.

    Parameters
    ----------
    model_hash : str
        The 'model_id_hash' value to look up.
    base_dir : str | Path
        Directory holding the rolling CSV parts and index.
    file_prefix : str
        Prefix of the rolling log.

    Returns
    -------
    bool
        True if present in the index; False otherwise.
    """
    idx = _read_index(_index_path(Path(base_dir), file_prefix))
    if idx.empty or "model_id_hash" not in idx.columns:
        return False
    return str(model_hash) in idx["model_id_hash"].astype("string").values

In [4]:
def _list_part_files(
    base_dir: Path,
    file_prefix: str,
    ext: str = "csv",
) -> list[Path]:
    """
    List existing rolling CSV parts for a given prefix, sorted by numeric part index.

    Parameters
    ----------
    base_dir : Path
        Directory to search.
    file_prefix : str
        Prefix of the rolling CSV set (e.g., 'marginal_emissions_log').
    ext : str, default 'csv'
        File extension (without dot).

    Returns
    -------
    list[Path]
        Sorted list of matching part files, e.g. [.../prefix.part000.csv, .../prefix.part001.csv, ...]
    """
    if not base_dir.exists():
        return []

    rx = re.compile(rf"^{re.escape(file_prefix)}\.part(\d+)\.{re.escape(ext)}$")
    parts: list[tuple[int, Path]] = []

    for p in base_dir.glob(f"{file_prefix}.part*.{ext}"):
        if not p.is_file():
            continue
        m = rx.match(p.name)
        if m:
            parts.append((int(m.group(1)), p))

    parts.sort(key=lambda t: t[0])
    return [p for _, p in parts]


In [5]:
def load_all_logs_rotating_csv(
    results_dir: str | Path = ".",
    file_prefix: str = "marginal_emissions_log",
) -> pd.DataFrame:
    """
    Read only parts referenced by the index; drop duplicate hashes (keep last).

    Parameters
    ----------
    results_dir: str | Path
        The directory containing the results.
    file_prefix: str
        The prefix of the log files to read.

    Returns
    -------
    pd.DataFrame
        The concatenated DataFrame containing the logs.
    """
    # Read the index file
    base_dir = Path(results_dir)
    idx = _read_index(_index_path(base_dir, file_prefix))
    # Check if the index is empty
    if idx.empty:
        return pd.DataFrame()
    # Get the unique parts to read
    parts = idx["part_file"].unique().tolist()
    # Read the parts into DataFrames
    dfs = [pd.read_csv(p) for p in parts if Path(p).exists()]
    # Check if any DataFrames were read
    if not dfs:
        return pd.DataFrame()
    # Concatenate the DataFrames
    out = pd.concat(dfs, ignore_index=True)
    # Drop duplicate model_id_hash entries
    if "model_id_hash" in out.columns:
        out = out.drop_duplicates(subset=["model_id_hash"], keep="last")
    return out


In [6]:
def _read_index(index_path: Path) -> pd.DataFrame:
    """
    Read the rolling-log index CSV (id→part mapping).

    Parameters
    ----------
    index_path : Path
        Path to '<file_prefix>_index.csv'.

    Returns
    -------
    pd.DataFrame
        Columns ['model_id_hash','part_file'] or empty frame if not found/invalid.
    """
    try:
        idx = pd.read_csv(index_path, dtype={"model_id_hash": "string", "part_file": "string"})
        if not {"model_id_hash","part_file"}.issubset(idx.columns):
            raise ValueError("Index missing required columns.")
        return idx
    except FileNotFoundError:
        return pd.DataFrame(columns=["model_id_hash","part_file"])
    except Exception:
        # Be permissive but return the expected schema
        return pd.DataFrame(columns=["model_id_hash","part_file"])


In [7]:
def remove_model_from_rotating_csv(
        model_hash: str,
        results_dir: str | Path = ".",
        file_prefix: str = "marginal_emissions_log",
) -> None:
    """
    Remove all rows with `model_id_hash == model_hash` from the rolling CSV set.

    Parameters
    ----------
    model_hash : str
        Identifier to remove.
    results_dir : str | Path, default "."
        Directory holding parts and index.
    file_prefix : str, default "marginal_emissions_log"
        Prefix of the rolling log files.
    """
    base_dir = _ensure_dir(Path(results_dir))
    idx_path = _index_path(base_dir, file_prefix)

    # Lock the index for the whole operation to avoid races with concurrent writers/readers
    with _file_lock(_index_lock_path(idx_path)):
        idx = _read_index(idx_path)
        if idx.empty:
            return

        # Drop from referenced part files
        for pf in idx.loc[idx["model_id_hash"] == model_hash, "part_file"].dropna().unique():
            _drop_hash_from_part(Path(pf), model_hash)

        # Update index
        idx = idx[idx["model_id_hash"] != model_hash]
        idx.to_csv(idx_path, index=False)


In [8]:
def save_summary_to_rotating_csv(
        summary_df: pd.DataFrame,
        results_dir: str | Path = ".",
        file_prefix: str = "marginal_emissions_log",
        max_mb: int = 95,
        force_overwrite: bool = False,
        naming: PartNaming | None = None,
        fsync: bool = False,
) -> Path:
    """
    Append a single-row summary to a rolling CSV (<prefix>.partNNN.csv) with strict rotation:
    - Per-file lock during append (prevents interleaved writes/duplicate headers)
    - Under-lock preflight ensures the write will NOT push the file over `max_mb`
      (allocates a new shard if necessary)
    - Atomic index update under lock

    Parameters
    ----------
    summary_df : pd.DataFrame
        Single-row DataFrame with at least a 'model_id_hash' column.
    results_dir : str | Path, default "."
        Directory to write parts and the index into.
    file_prefix : str, default "marginal_emissions_log"
        Prefix of the part files ('<prefix>.partNNN.csv').
    max_mb : int, default 95
        Rotate when current part would exceed this size (MiB) after the append.
    force_overwrite : bool, default False
        If True, delete existing rows with the same hash before appending.
    naming : PartNaming, optional
        Naming convention (token/width/ext). If provided, `ext` should include the dot
        (e.g., ".csv"). Internally we use the extension without the dot for matching.
    fsync : bool, default False
        If True, call fsync() on the file after writing to ensure data is flushed to disk.

    Returns
    -------
    Path
        The part file path that received the append.

    Raises
    ------
    ValueError
        If `summary_df` is empty or missing 'model_id_hash'.
    """
    if summary_df.empty:
        raise ValueError("summary_df is empty.")
    if "model_id_hash" not in summary_df.columns:
        raise ValueError("summary_df must contain 'model_id_hash'.")
    if len(summary_df) != 1:
        summary_df = summary_df.iloc[:1].copy()

    naming = naming or PartNaming()
    base_dir = _ensure_dir(Path(results_dir))
    idx_path = _index_path(base_dir, file_prefix)
    model_hash = str(summary_df["model_id_hash"].iloc[0])
    ext_nodot = naming.ext.lstrip(".")

    # Optional overwrite: remove old rows (parts + index)
    if force_overwrite:
        remove_model_from_rotating_csv(model_hash, base_dir, file_prefix)
    else:
        if is_model_logged_rotating_csv(model_hash, base_dir, file_prefix):
            print(f"[SKIP] Hash already indexed: {model_hash}")
            parts = _list_part_files(base_dir, file_prefix, ext=ext_nodot)
            return parts[-1] if parts else base_dir / naming.format(file_prefix, 0)

    # Determine candidate shard
    parts = _list_part_files(base_dir, file_prefix, ext=ext_nodot)
    if parts:
        target = parts[-1]
    else:
        target = allocate_next_part(base_dir, file_prefix, width=naming.width, ext=ext_nodot)

    threshold_bytes = int(max_mb * 1024 * 1024)

    # --- LOCK AND WRITE TO SHARD SAFELY ---
    while True:
        shard_lock = Path(str(target) + ".lock")
        with _file_lock(shard_lock):
            current_size = Path(target).stat().st_size if Path(target).exists() else 0
            write_header = (current_size == 0)
            csv_payload = summary_df.to_csv(index=False, header=write_header)
            payload_bytes = len(csv_payload.encode("utf-8"))

            if current_size + payload_bytes > threshold_bytes:
                # rotate: leave lock, allocate new shard, try again
                pass
            else:
                with open(target, "a", encoding="utf-8", newline="") as f:
                    f.write(csv_payload)
                    f.flush()
                    if fsync:
                        os.fsync(f.fileno())
                break

        target = allocate_next_part(base_dir, file_prefix, width=naming.width, ext=ext_nodot)

    # --- LOCK AND UPDATE INDEX (atomic replace + optional fsync) ---
    lock_path = _index_lock_path(idx_path)
    with _file_lock(lock_path):
        idx = _read_index(idx_path)
        already = ("model_id_hash" in idx.columns) and (model_hash in idx["model_id_hash"].astype("string").values)
        if not already:
            idx = pd.concat(
                [idx, pd.DataFrame([{"model_id_hash": model_hash, "part_file": str(target)}])],
                ignore_index=True,
            )
            tmp_idx = idx_path.with_suffix(idx_path.suffix + ".tmp")
            with open(tmp_idx, "w", encoding="utf-8", newline="") as fh:
                idx.to_csv(fh, index=False)
                fh.flush()
                if fsync:
                    os.fsync(fh.fileno())
            os.replace(tmp_idx, idx_path)
            if fsync:
                # Ensure directory entry for index is durable
                dir_fd = os.open(str(idx_path.parent), os.O_DIRECTORY)
                try:
                    os.fsync(dir_fd)
                finally:
                    os.close(dir_fd)

    print(f"[SAVE] Appended to {target}, index updated.")
    return target

#### General

In [9]:
def _file_size_mb(path: Path) -> float:
    """
    Return size of `path` in MiB. If file doesn't exist, returns 0.0.

    Parameters
    ----------
    path : Path
        Path to the file.

    Returns
    -------
    float
        Size of the file in MiB.
    """
    p = Path(path)
    if not p.exists():
        return 0.0
    return p.stat().st_size / (1024 * 1024.0)


#### Logging

In [10]:
def load_existing_hashes(
        results_dir: str | Path,
        file_prefix: str,
) -> set[str]:
    """
    Get all unique `model_id_hash` values from the rolling-log index.

    Parameters
    ----------
    results_dir : str | Path
        Directory containing the rolling CSV parts and index.
    file_prefix : str
        Prefix of the rolling log files.

    Returns
    -------
    set[str]
        Unique model_id_hash values present in the index.
    """
    idx = _read_index(_index_path(Path(results_dir), file_prefix))
    if idx.empty or "model_id_hash" not in idx.columns:
        return set()
    # Ensure NA is dropped and cast to Python strings
    return set(idx["model_id_hash"].dropna().astype(str).tolist())


In [11]:
def make_config_key(
        config: Mapping[str, Any],
          algo: str = "sha256"
) -> str:
    """
    Create a deterministic hash key for a configuration mapping.

    Parameters
    ----------
    config : Mapping[str, Any]
        Configuration to serialize. Keys should be stringable.
    algo : {'sha256','md5','sha1',...}, default 'sha256'
        Hash algorithm name passed to hashlib.new.

    Returns
    -------
    str
        Hex digest of the normalized, JSON-serialized configuration.
    """
    def _norm(x):
        # Order/JSON-stable normalization.
        if isinstance(x, Mapping):
            # sort by key string to be robust to non-string keys
            return {str(k): _norm(v) for k, v in sorted(x.items(), key=lambda kv: str(kv[0]))}
        if isinstance(x, (list, tuple)):
            return [_norm(v) for v in x]
        if isinstance(x, set):
            # sets are unordered; sort normalized elements
            return sorted(_norm(v) for v in x)
        if isinstance(x, (np.floating, np.integer, np.bool_)):
            return x.item()
        if isinstance(x, (datetime,)):
            return x.isoformat()
        return x  # strings, ints, floats, bools, None, etc.

    payload = json.dumps(
        _norm(config),
        sort_keys=True,
        separators=(",", ":"),
        ensure_ascii=False,
        default=str,   # last-resort for odd objects
    )
    h = hashlib.new(algo)
    h.update(payload.encode("utf-8"))
    return h.hexdigest()

In [12]:
def signature_for_run(
        user_pipeline: Pipeline,
        x_columns: list[str],
        y: pd.Series | pd.DataFrame,
        *,
        random_state: int,
        eval_splits: tuple[str, ...] = ("train", "validation"),
        compute_test: bool = False,
        extra_info: dict | None = None,
) -> tuple[str, dict]:
    """
    Build a stable config mapping for a model run and return (hash_key, mapping).

    This just standardizes what goes into the signature so different call sites
    don’t accidentally diverge.

    Parameters
    ----------
    user_pipeline : Pipeline
        The user-defined pipeline to run.
    x_columns : list[str]
        The feature columns to use for the model.
    y : pd.Series | pd.DataFrame
        The target variable(s) for the model.
    random_state : int
        The random seed to use for the model.
    eval_splits : tuple[str, ...], default=("train", "validation")
        The data splits to evaluate the model on.
    compute_test : bool, default=False
        Whether to compute metrics on the test split.
    extra_info : dict | None, default=None
        Any extra information to include in the signature.

    Returns
    -------
    tuple[str, dict]
        The hash key and the signature mapping.
    """
    sig = {
        "pipeline_params": user_pipeline.get_params(deep=True),
        "x_columns": list(x_columns),
        "y_columns": _y_columns_for_signature(y),
        "random_state": int(random_state),
        "eval_splits": tuple(eval_splits),
        "compute_test": bool(compute_test),
        **(extra_info or {}),
    }
    return make_config_key(sig), sig

In [13]:
def _y_columns_for_signature(y: pd.Series | pd.DataFrame) -> list[str]:
    """
    Normalize y to a list of column names for signature purposes.

    Parameters
    ----------
    y : pd.Series | pd.DataFrame
        The target variable(s) for the model.

    Returns
    -------
    list[str]
        The list of column names for the target variable(s).
    """
    if isinstance(y, pd.DataFrame):
        if y.shape[1] != 1:
            raise ValueError("y must be a Series or single-column DataFrame for signature.")
        return [str(y.columns[0])]
    name = getattr(y, "name", None)
    return [str(name)] if name is not None else ["y"]


#### MPI Management

In [14]:
def allocate_next_part(
        base_dir: Path,
        file_prefix: str,
        width: int = 3,
        ext: str = "csv",
        max_retries: int = 32,
        jitter_ms: tuple[int, int] = (1, 40),
) -> Path:
    """
    Atomically allocate the next rotating part file by creating it exclusively.

    Uses os.open(..., O_CREAT|O_EXCL) so only one process can create a given part.
    If another process wins the race, we re-scan and try the next part number.

    Parameters
    ----------
    base_dir : Path
        Directory to write part files into (created if missing).
    file_prefix : str
        Prefix used before ".partNNN.<ext>".
    width : int, default 3
        Minimum zero-padding for part numbers if none exist.
    ext : str, default "csv"
        Extension without dot.
    max_retries : int, default 32
        Maximum attempts before giving up.
    jitter_ms : (int, int), default (1, 40)
        Random backoff (min,max) milliseconds between retries.

    Returns
    -------
    Path
        The newly created, zero-length part file path (claimed for you).

    Raises
    ------
    RuntimeError
        If a unique part file cannot be allocated within max_retries.
    """
    base_dir = Path(base_dir)
    base_dir.mkdir(parents=True, exist_ok=True)

    for _ in range(max_retries):
        path = _next_csv_part_path(base_dir, file_prefix, width=width, ext=ext)
        flags = os.O_CREAT | os.O_EXCL | os.O_WRONLY
        try:
            fd = os.open(path, flags)  # atomic claim
            os.close(fd)               # leave it for normal open() later
            return path
        except FileExistsError:
            # Someone else grabbed it; small random backoff, then try again
            time.sleep(random.uniform(*jitter_ms) / 1000.0)
            continue

    raise RuntimeError("Failed to allocate a unique part file after many attempts")


In [15]:
def _distribute_configs(
        configs: list[dict],
        rank: int,
        size: int,
        mode: str = "stride"
) -> list[dict]:
    """
    Distribute configurations across multiple ranks.

    Parameters
    ----------
    configs: list[dict]
        The list of configurations to distribute.
    rank: int
        The rank of the current process.
    size: int
        The total number of processes.
    mode: str
        The distribution mode ("stride" or "chunked").

    Returns
    -------
    list[dict]
        The distributed list of configurations.
    """
    # Handle single process case
    if size <= 1:
        return configs
    # Handle multi-process case
    if mode == "stride":
        return configs[rank::size]
    # chunked
    n = len(configs)
    start = (n * rank) // size
    end   = (n * (rank + 1)) // size
    return configs[start:end]

In [16]:
@contextmanager
def _file_lock(lock_path: Path, max_wait_s: float = 30.0, jitter_ms: tuple[int,int]=(2,25)):
    """
    Simple cross-process lock using O_CREAT|O_EXCL on a lockfile.

    Parameters
    ----------
    lock_path : Path
        Path to the lock file to create.
    max_wait_s : float, default 30.0
        Maximum time to wait for the lock before raising TimeoutError.
    jitter_ms : (int,int), default (2,25)
        Randomized backoff between retries, in milliseconds.

    Yields
    ------
    None
        The lock is held for the duration of the context.
    """
    # Create the lock file
    deadline = time.time() + float(max_wait_s)
    lock_path = Path(lock_path)
    last_err = None
    # Wait for the lock to be available
    while time.time() < deadline:
        try:
            fd = os.open(lock_path, os.O_CREAT | os.O_EXCL | os.O_WRONLY)
            os.close(fd)
            try:
                yield
            finally:
                try:
                    os.unlink(lock_path)
                except FileNotFoundError:
                    pass
            return
        except FileExistsError as e:
            last_err = e
            time.sleep(random.uniform(*jitter_ms) / 1000.0)
    raise TimeoutError(f"Could not acquire lock: {lock_path}") from last_err

In [17]:
def _mpi_context():
    """
    Get the MPI context for distributed training.

    Returns
    -------
    Tuple[COMM, int, int]
        The MPI communicator, rank, and size.
    """
    try:
        from mpi4py import MPI  # ensures import
        comm = MPI.COMM_WORLD
        return comm, comm.Get_rank(), comm.Get_size()
    except Exception:
        class _Dummy:  # single-process stub
            def bcast(self, x, root=0): return x
            def Barrier(self): pass
        return _Dummy(), 0, 1

#### Naming Conventions

In [18]:
@dataclass(frozen=True)
class PartNaming:
    token: str = ".part"   # separator between stem and index
    width: int = 3         # zero-pad width
    ext: str = ".csv"      # file extension, with leading dot

    def format(self,
            stem: str,
            idx: int
    ) -> str:
        """
        Format a part filename.

        Parameters
        ----------
        stem : str
            The base name of the file (without extension or part token).
        idx : int
            The part index (zero-padded).

        Returns
        -------
        str
            The formatted part filename.
        """
        return f"{stem}{self.token}{idx:0{self.width}d}{self.ext}"

    def split(self,
            name: str
    ) -> Tuple[str, int | None]:
        """
        Split a part filename into its stem and index.

        Parameters
        ----------
        name : str
            The part filename to split.

        Returns
        -------
        Tuple[str, int | None]
            The stem and index of the part filename.
        """
        # returns (stem, idx) where idx is None if no part index present
        if not name.endswith(self.ext):
            # unknown extension; treat everything before first '.' as stem
            p = Path(name)
            return (p.stem, None)
        base = name[: -len(self.ext)]
        if self.token in base:
            stem, idx_str = base.split(self.token, 1)
            if idx_str.isdigit():
                return stem, int(idx_str)
        return base, None


#### Path and Directory Management

In [19]:
def _ensure_dir(
        d: str | Path,
        *,
        resolve: bool = True
) -> Path:
    """
    Ensure directory `d` exists and return it as a Path.

    - Creates parent directories as needed.
    - Raises a clear error if a non-directory already exists at `d`.
    - Optionally returns the resolved (absolute) path.

    Parameters
    ----------
    d : str | Path
        Directory path to create if missing.
    resolve : bool, default True
        If True, return Path.resolve(strict=False) to normalize/absolutize.

    Returns
    -------
    Path
        The (optionally resolved) directory path.
    """
    p = Path(d)
    if p.exists() and not p.is_dir():
        raise NotADirectoryError(f"Path exists and is not a directory: {p}")
    p.mkdir(parents=True, exist_ok=True)
    return p.resolve(strict=False) if resolve else p


In [20]:
def _index_lock_path(index_path: Path) -> Path:
    """
    Derive the lock file path for an index CSV (same directory, '.lock' suffix).

    Parameters
    ----------
    index_path : Path
        Path to the index CSV file.

    Returns
    -------
    Path
        Path to the lock file.
    """
    return index_path.with_suffix(index_path.suffix + ".lock")

In [21]:
def _index_path(
        base_dir: Path,
        file_prefix: str
) -> Path:
    """
    Build the path to the global index CSV for a given rolling log set.

    Parameters
    ----------
    base_dir : Path
        Directory that holds the rolling CSV parts.
    file_prefix : str
        Prefix used by the rolling CSV (e.g., 'marginal_emissions_log').

    Returns
    -------
    Path
        '<base_dir>/<file_prefix>_index.csv'
    """
    return Path(base_dir) / f"{file_prefix}_index.csv"


In [22]:
def _next_csv_part_path(base_dir: Path, file_prefix: str, width: int = 3, ext: str = "csv") -> Path:
    """
    Return the next available rotating-CSV part path.

    Scans for files named "<file_prefix>.partNNN.<ext>" in `base_dir`, where NNN is an
    integer with zero-padding. Picks max(N) and returns the next. If none exist, returns
    "...part000.<ext>" (or the padding width you pass).

    Parameters
    ----------
    base_dir : Path
        Directory to scan for part files.
    file_prefix : str
        Prefix used before ".partNNN.<ext>".
    width : int, default 3
        Minimum zero-padding width if no files exist yet.
    ext : str, default "csv"
        File extension (without dot).

    Returns
    -------
    Path
        Path for the next part file (not created).
    """
    if width < 1:
        raise ValueError("width must be >= 1")

    base_dir = Path(base_dir)
    pattern = re.compile(rf"^{re.escape(file_prefix)}\.part(\d+)\.{re.escape(ext)}$")

    max_n = -1
    pad = width

    for p in base_dir.glob(f"{file_prefix}.part*.{ext}"):
        m = pattern.match(p.name)
        if not m:
            continue
        n_str = m.group(1)
        pad = max(pad, len(n_str))
        try:
            n = int(n_str)
        except ValueError:
            continue
        if n > max_n:
            max_n = n

    next_n = max_n + 1
    n_str = f"{next_n:0{pad}d}"
    return base_dir / f"{file_prefix}.part{n_str}.{ext}"

In [23]:
def _roll_if_needed(
        path: Path,
        max_mb: int,
        *,
        naming: PartNaming | None = None
) -> Path:
    """
    If `path` exists and is >= max_mb, return the *next* part filename.
    Otherwise return `path` unchanged.

    Parameters
    ----------
    path : Path
        Current part file path (e.g., 'prefix.part007.csv').
    max_mb : int
        Rotation threshold in mebibytes (MiB).
    naming : PartNaming, optional
        Naming convention (token/width/ext). Uses defaults if not provided.

    Returns
    -------
    Path
        Either `path` or a new sibling with incremented part index.
    """
    if not path.exists() or _file_size_mb(path) < float(max_mb):
        return path
    naming = naming or PartNaming()
    stem, idx = naming.split(path.name)
    next_idx = (idx or 0) + 1
    return path.with_name(naming.format(stem=stem, idx=next_idx))


#### Scoring & Metrics

In [24]:
def _compute_group_energy_weights(
    df: pd.DataFrame,
    group_col: str,
    q_col: str,
    interval_hours: float = 0.5,
) -> pd.DataFrame:
    """
    Aggregate energy weights by group.

    Parameters
    ----------
    df : pd.DataFrame
        Rows for a single split after preprocessing (must contain `group_col` and `q_col`).
    group_col : str
        Name of the group id column (e.g., 'median_group_id', 'quantile_group_id').
    q_col : str
        Name of the demand/quantity column used as Q in the regression (usually x_vars[0]).
    interval_hours : float, default 0.5
        Duration represented by each row in hours (half-hourly = 0.5).

    Returns
    -------
    pd.DataFrame
        Columns: [group_col, 'q_sum', 'energy_MWh']
        where energy_MWh = q_sum * interval_hours.
    """
    if group_col not in df.columns:
        raise KeyError(f"'{group_col}' not found in df")
    if q_col not in df.columns:
        raise KeyError(f"'{q_col}' not found in df")
    if not np.issubdtype(np.asarray(df[q_col]).dtype, np.number):
        raise TypeError(f"'{q_col}' must be numeric")
    if interval_hours <= 0:
        raise ValueError("interval_hours must be > 0")

    g = (
        df.groupby(group_col, observed=True)[q_col]
          .sum()
          .rename("q_sum")
          .reset_index()
    )
    g["energy_MWh"] = g["q_sum"] * float(interval_hours)
    return g


In [25]:
def finite_difference_me_metrics(
    df: pd.DataFrame,
    time_col: str = "timestamp",
    q_col: str = "demand_met",
    y_col: str = "tons_co2",
    me_col: str = "ME",
    group_keys: list[str] | tuple[str, ...] = ("city",),
    max_dt: pd.Timedelta = pd.Timedelta("2h"),
    min_abs_dq: float = 1e-6,
) -> pd.DataFrame:
    """
    Compare predicted ME to observed short-horizon slopes s = Δy/ΔQ on held-out data.

    For each group in `group_keys`:
      Δy = y_t - y_{t-1}, ΔQ = Q_t - Q_{t-1}, Δt = t - t_{t-1}
      Keep pairs with Δt ≤ max_dt and |ΔQ| ≥ min_abs_dq.
      s_t = Δy / ΔQ, ME_avg = 0.5*(ME_t + ME_{t-1})

    Returns
    -------
    pd.DataFrame
        One row per group and an optional pooled 'ALL' row:
        ['pearson_r','spearman_r','rmse','mae','n_pairs', *group_keys]
    """
    if time_col not in df.columns:
        raise KeyError(f"'{time_col}' not in df")
    # ensure datetime for Δt filtering
    dt_series = pd.to_datetime(df[time_col], errors="coerce")
    if dt_series.isna().any():
        raise ValueError(f"Column '{time_col}' contains non-parseable datetimes")
    work = df.copy()
    work[time_col] = dt_series

    def _per_group(gdf: pd.DataFrame) -> dict:
        gdf = gdf.sort_values(time_col).copy()
        gdf["dt"] = gdf[time_col].diff()
        gdf["dQ"] = gdf[q_col].diff()
        gdf["dY"] = gdf[y_col].diff()
        gdf["ME_avg"] = 0.5 * (gdf[me_col] + gdf[me_col].shift(1))

        mask = (
            gdf["dt"].notna() & (gdf["dt"] <= max_dt)
            & gdf["dQ"].notna() & (np.abs(gdf["dQ"]) >= float(min_abs_dq))
            & gdf["dY"].notna() & gdf["ME_avg"].notna()
        )
        sub = gdf.loc[mask, ["dY", "dQ", "ME_avg"]]
        if sub.empty:
            return {"pearson_r": np.nan, "spearman_r": np.nan, "rmse": np.nan, "mae": np.nan, "n_pairs": 0}

        s = sub["dY"].to_numpy(dtype=float) / sub["dQ"].to_numpy(dtype=float)
        me = sub["ME_avg"].to_numpy(dtype=float)
        return {
            "pearson_r": float(pd.Series(s).corr(pd.Series(me))),
            "spearman_r": float(pd.Series(s).corr(pd.Series(me), method="spearman")),
            "rmse": float(root_mean_squared_error(s, me)),
            "mae": float(mean_absolute_error(s, me)),
            "n_pairs": int(len(sub)),
        }

    parts: list[dict] = []
    if group_keys:
        for keys, gdf in work.groupby(list(group_keys), observed=True, sort=True):
            row = _per_group(gdf)
            if isinstance(keys, tuple):
                for kname, kval in zip(group_keys, keys):
                    row[kname] = kval
            else:
                row[group_keys[0]] = keys
            parts.append(row)
    else:
        parts.append(_per_group(work) | {"group": "ALL"})

    out = pd.DataFrame(parts)

    # pooled row
    if group_keys and (not out.empty) and out["n_pairs"].sum() > 0:
        tmp = []
        for _, gdf in work.groupby(list(group_keys), observed=True, sort=True):
            gdf = gdf.sort_values(time_col).copy()
            gdf["dt"] = gdf[time_col].diff()
            gdf["dQ"] = gdf[q_col].diff()
            gdf["dY"] = gdf[y_col].diff()
            gdf["ME_avg"] = 0.5 * (gdf[me_col] + gdf[me_col].shift(1))
            mask = (
                gdf["dt"].notna() & (gdf["dt"] <= max_dt)
                & gdf["dQ"].notna() & (np.abs(gdf["dQ"]) >= float(min_abs_dq))
                & gdf["dY"].notna() & gdf["ME_avg"].notna()
            )
            sub = gdf.loc[mask, ["dY", "dQ", "ME_avg"]]
            if not sub.empty:
                tmp.append(
                    pd.DataFrame({
                        "s": sub["dY"].to_numpy(dtype=float) / sub["dQ"].to_numpy(dtype=float),
                        "ME_avg": sub["ME_avg"].to_numpy(dtype=float),
                    })
                )
        if tmp:
            pooled = pd.concat(tmp, ignore_index=True)
            pooled_row = {
                "pearson_r": float(pooled["s"].corr(pooled["ME_avg"])),
                "spearman_r": float(pooled["s"].corr(pooled["ME_avg"], method="spearman")),
                "rmse": float(root_mean_squared_error(pooled["s"], pooled["ME_avg"])),
                "mae": float(mean_absolute_error(pooled["s"], pooled["ME_avg"])),
                "n_pairs": int(len(pooled)),
            }
            for k in group_keys:
                pooled_row[k] = "ALL"
            out = pd.concat([out, pd.DataFrame([pooled_row])], ignore_index=True)

    return out


In [26]:
def macro_micro_means(df: pd.DataFrame, metric: str, weight_col: str = "n_obs") -> dict:
    """
    Compute macro (simple mean) and micro (weighted by `weight_col`) for a metric.

    Parameters
    ----------
    df : pd.DataFrame
        Per-group metrics.
    metric : str
        Column name to average.
    weight_col : str, default "n_obs"
        Column to use as weights for micro average.

    Returns
    -------
    dict
        {"macro": float, "micro": float}
    """
    macro = float(np.nanmean(df[metric].to_numpy(dtype=float)))
    if (weight_col in df) and np.nansum(df[weight_col].to_numpy(dtype=float)) > 0:
        micro = float(np.average(df[metric], weights=df[weight_col]))
    else:
        micro = np.nan
    return {"macro": macro, "micro": micro}


In [27]:
def mean_absolute_percentage_error(
        y_true,
        y_pred,
        eps: float = 1e-6
) -> float:
    """
    Compute MAPE robustly - adding small constant to avoid division by zero.

    MAPE = mean(|(y_true - y_pred) / (|y_true| + eps)|) * 100

    Parameters
    ----------
    y_true : array-like
        Ground-truth values.
    y_pred : array-like
        Predicted values.
    eps : float, default 1e-6
        Small constant to avoid division by zero.

    Returns
    -------
    float
        Mean absolute percentage error in percent.
    """
    # true values for y
    yt = np.asarray(y_true, dtype=float)
    # predicted values for y
    yp = np.asarray(y_pred, dtype=float)
    # denominator
    denom = np.abs(yt) + float(eps)
    # compute MAPE
    m = np.abs((yt - yp) / denom)
    # return as percentage (*100)
    return float(np.nanmean(m) * 100.0)


In [28]:
def mean_metric(df: pd.DataFrame, metric: str) -> float:
    """
    Compute the mean of a metric, with a special case for MSE derived from RMSE.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing metric columns.
    metric : {"r2","rmse","mae","mape","n_obs","mse"}
        Metric to aggregate.

    Returns
    -------
    float
        NaN-safe mean of the requested metric.

    Raises
    ------
    KeyError
        If required columns are missing.
    """
    if metric == "mse":
        if "rmse" not in df:
            raise KeyError("Cannot compute 'mse': 'rmse' column missing.")
        return float(np.nanmean(df["rmse"].to_numpy(dtype=float) ** 2))
    if metric not in df:
        raise KeyError(f"Metric '{metric}' not found in DataFrame.")
    return float(np.nanmean(df[metric].to_numpy(dtype=float)))


In [29]:
def pooled_co2_metrics(
    regressor,                  # fitted GroupwiseRegressor
    transformed_df: pd.DataFrame,
    y_col: str | None = None,
    group_col: str | None = None,
) -> dict:
    """
    Compute pooled (all bins together) out-of-sample metrics for CO2.

    Parameters
    ----------
    regressor : GroupwiseRegressor
        Must be fitted; `regressor.group_models_` is used per group.
    transformed_df : pd.DataFrame
        Contains features used by the regressor, the group column, and the true y.
        (Typically validation/test X after feature+binner, with y added).
    y_col : str, optional
        Target column name. Defaults to regressor.y_var.
    group_col : str, optional
        Group column name. Defaults to regressor.group_col.

    Returns
    -------
    dict
        {'r2','rmse','mae','mape','n_obs'} (NaNs if insufficient data).
    """
    y_col = y_col or regressor.y_var
    group_col = group_col or regressor.group_col
    if y_col not in transformed_df.columns:
        raise KeyError(f"'{y_col}' not found in transformed_df")
    if group_col not in transformed_df.columns:
        raise KeyError(f"'{group_col}' not found in transformed_df")

    preds = pd.Series(index=transformed_df.index, dtype=float)
    for g, gdf in transformed_df.groupby(group_col, sort=True):
        model = regressor.group_models_.get(g)
        if model is None:
            continue
        preds.loc[gdf.index] = model.predict(gdf)

    mask = preds.notna()
    n_obs = int(mask.sum())
    if n_obs == 0:
        return {"r2": np.nan, "rmse": np.nan, "mae": np.nan, "mape": np.nan, "n_obs": 0}

    y_true = transformed_df.loc[mask, y_col].to_numpy(dtype=float)
    y_pred = preds.loc[mask].to_numpy(dtype=float)

    # r2 can error for <2 samples or constant y
    try:
        r2 = float(r2_score(y_true, y_pred))
    except Exception:
        r2 = np.nan

    return {
        "r2": r2,
        "rmse": float(root_mean_squared_error(y_true, y_pred)),
        "mae": float(mean_absolute_error(y_true, y_pred)),
        "mape": float(mean_absolute_percentage_error(y_true, y_pred)),
        "n_obs": n_obs,
    }

In [30]:
def summarise_metrics_logs(
        train_logs: pd.DataFrame,
        val_logs: pd.DataFrame,
        test_logs: pd.DataFrame | None = None,
        user_pipeline: Pipeline = None,
        x_columns: list | None = None,
        random_state: int = 12,
        group_col_name: str = "group",
        pooled_metrics_by_split: dict[str, dict] | None = None,
        fd_me_metrics_by_split: dict[str, dict] | None = None,
        energy_weight_col: str = "energy_MWh",
) -> pd.DataFrame:
    """
    Summarise per-split, per-group metrics and pipeline metadata into a single-row DataFrame.

    This variant allows `test_logs` to be None (can skip test during tuning).

    Parameters
    ----------
    train_logs, val_logs : pd.DataFrame
        Metrics frames for train/validation.
    test_logs : pd.DataFrame or None, default None
        Test metrics; if None, test columns are omitted from the summary.
    user_pipeline : Pipeline
        The fitted or configured pipeline (used for metadata).
    x_columns : list, optional
        Feature names used by the model.
    random_state : int, default 12
        Random seed to record.
    group_col_name : str, default "group"
        Canonical name for the group column.
    pooled_metrics_by_split, fd_me_metrics_by_split : dict, optional
        Optional extra diagnostics keyed by split.
    energy_weight_col : str, default "energy_MWh"
        Column name to use for energy-weighted micro-averages if present.

    Returns
    -------
    pd.DataFrame
        One-row summary. Only includes split columns for the splits provided.
    """
    def _norm(df: pd.DataFrame) -> pd.DataFrame:
        if df is None or df.empty:
            return df

        cols = list(df.columns)

        # If desired already present, use it
        if group_col_name in cols:
            return df

        # If a plain 'group' exists, rename it to the desired name
        if "group" in cols:
            return df.rename(columns={"group": group_col_name})

        # Known aliases we can rename from
        candidates = [
            "multi_group_id",
            "quantile_group_id",
            "median_group_id",
            "original_quantile_group_id",
            "group_id",
        ]

        # Any *_group_id pattern
        pattern_hits = [c for c in cols if c.endswith("_group_id")]

        # Prefer known aliases in order
        for c in candidates:
            if c in cols:
                return df.rename(columns={c: group_col_name})

        # If exactly one *_group_id exists, use it
        if len(pattern_hits) == 1:
            return df.rename(columns={pattern_hits[0]: group_col_name})

        # Nothing we recognize → fail loudly with context
        raise KeyError(
            f"Could not locate a group column; expected '{group_col_name}' or any of "
            f"{[c for c in candidates if c in cols] + (['group'] if 'group' in cols else []) or candidates + ['group']}. "
            f"Available columns: {cols}"
        )
    splits: dict[str, pd.DataFrame] = {
        "train": _norm(train_logs.copy()),
        "validation": _norm(val_logs.copy()),
    }
    if test_logs is not None:
        splits["test"] = _norm(test_logs.copy())

    required = {"r2", "rmse", "mae", "mape", "n_obs"}
    for name, df in splits.items():
        missing = required.difference(df.columns)
        if missing:
            raise ValueError(f"{name} logs missing metrics: {sorted(missing)}")

    first = next(iter(splits.values()))
    model_id = first.get("model_id_hash", pd.Series([np.nan])).iloc[0]
    log_time = first.get("log_time", pd.Series([np.nan])).iloc[0]
    model_name = user_pipeline._final_estimator.__class__.__name__ if user_pipeline is not None else ""
    pipeline_steps = list(user_pipeline.named_steps.keys()) if user_pipeline is not None else []

    summary: dict[str, Any] = {
        "model_id_hash": model_id,
        "random_state": random_state,
        "params_json": json.dumps(
            user_pipeline.get_params(deep=True), sort_keys=True, separators=(",", ":"), default=str
        ) if user_pipeline is not None else "{}",
        "log_time": log_time,
        "model_name": model_name,
        "pipeline_steps": pipeline_steps,
        "pipeline_n_steps": len(pipeline_steps),
        "x_columns": x_columns or [],
        "metrics_by_group": {},
    }

    nested: dict[str, dict] = {}
    for split, df in splits.items():
        # macro means
        summary[f"r2_{split}"] = float(df["r2"].mean())
        summary[f"rmse_{split}"] = float(df["rmse"].mean())
        summary[f"mae_{split}"] = float(df["mae"].mean())
        summary[f"mape_{split}"] = float(df["mape"].mean())
        # counts should be sums, not means
        summary[f"n_obs_{split}"] = int(df["n_obs"].sum())
        summary[f"mse_{split}"] = float((df["rmse"] ** 2).mean())

        # micro by n_obs
        if df["n_obs"].sum() > 0:
            w = df["n_obs"].to_numpy(dtype=float)
            summary[f"r2_{split}_micro"] = float(np.average(df["r2"], weights=w))
            summary[f"rmse_{split}_micro"] = float(np.average(df["rmse"], weights=w))
            summary[f"mae_{split}_micro"] = float(np.average(df["mae"], weights=w))
            summary[f"mape_{split}_micro"] = float(np.average(df["mape"], weights=w))
        else:
            summary[f"r2_{split}_micro"] = np.nan
            summary[f"rmse_{split}_micro"] = np.nan
            summary[f"mae_{split}_micro"] = np.nan
            summary[f"mape_{split}_micro"] = np.nan

        # energy-weighted micro (if provided)
        if (energy_weight_col in df.columns) and (df[energy_weight_col].fillna(0).sum() > 0):
            wE = df[energy_weight_col].fillna(0).to_numpy(dtype=float)
            summary[f"r2_{split}_energy_micro"] = float(np.average(df["r2"], weights=wE))
            summary[f"rmse_{split}_energy_micro"] = float(np.average(df["rmse"], weights=wE))
            summary[f"mae_{split}_energy_micro"] = float(np.average(df["mae"], weights=wE))
            summary[f"mape_{split}_energy_micro"] = float(np.average(df["mape"], weights=wE))
            summary[f"{energy_weight_col}_{split}_total"] = float(wE.sum())
        else:
            summary[f"r2_{split}_energy_micro"] = np.nan
            summary[f"rmse_{split}_energy_micro"] = np.nan
            summary[f"mae_{split}_energy_micro"] = np.nan
            summary[f"mape_{split}_energy_micro"] = np.nan
            summary[f"{energy_weight_col}_{split}_total"] = 0.0

        cols = ["r2", "rmse", "mae", "mape", "n_obs"]
        if energy_weight_col in df.columns:
            cols.append(energy_weight_col)
        nested[split] = df.set_index(group_col_name)[cols].to_dict(orient="index")

    summary["metrics_by_group"] = nested

    pooled_metrics_by_split = pooled_metrics_by_split or {}
    fd_me_metrics_by_split = fd_me_metrics_by_split or {}
    for split in splits.keys():
        summary[f"pooled_co2_{split}"] = json.dumps(pooled_metrics_by_split.get(split, {}))
        summary[f"fd_me_{split}"] = json.dumps(fd_me_metrics_by_split.get(split, {}))

    return pd.DataFrame([summary])

### Transformers / Classes 

#### Feature Engineering Transformers

In [31]:
class AnalysisFeatureAdder(BaseEstimator, TransformerMixin):
    """
    Add core temporal and quantitative features used in the original analysis.

    Adds:
      - time_id:              HH-MM string from `timestamp_col`
      - <Q>_sqrd:             square of `demand_met_col`
      - log_<Q>:              log(demand_met + ε)
      - log_<Q>_sqrd:         (log_<Q>)^2
      - log_<CO2>:            log(tons_co2 + ε) (only if `co2_col` present)
    """

    def __init__(
        self,
        timestamp_col: str = "timestamp",
        demand_met_col: str = "demand_met",
        co2_col: str = "tons_co2",
        epsilon: float = 1e-6,
    ):
        """
        Parameters
        ----------
        timestamp_col : str
            Name of the datetime column (parseable by pandas).
        demand_met_col : str
            Name of the demand column.
        co2_col : str
            Name of the CO2 column (optional at transform time).
        epsilon : float, default 1e-6
            Small constant to avoid log(0).
        """
        if not isinstance(timestamp_col, str):
            raise ValueError("timestamp_col must be a string")
        if not isinstance(demand_met_col, str):
            raise ValueError("demand_met_col must be a string")
        if not isinstance(co2_col, str):
            raise ValueError("co2_col must be a string")
        if not isinstance(epsilon, (float, int)):
            raise ValueError("epsilon must be a float or int")

        self.timestamp_col = timestamp_col
        self.demand_met_col = demand_met_col
        self.co2_col = co2_col
        self.epsilon = float(epsilon)

    def fit(self, X, y=None):
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input must be a pandas DataFrame")
        for col in [self.timestamp_col, self.demand_met_col]:
            if col not in X.columns:
                raise ValueError(f"Missing required column '{col}' in input DataFrame")
        self.n_features_in_ = X.shape[1]
        self.is_fitted_ = True
        return self

    def transform(self, X: pd.DataFrame, y: pd.Series | None = None) -> pd.DataFrame:
        """
        Parameters
        ----------
        X : pd.DataFrame
            Must contain `timestamp_col` and `demand_met_col`.

        Returns
        -------
        pd.DataFrame
            Copy of X with additional feature columns.
        """
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input must be a pandas DataFrame")

        df = X.copy()

        for col in [self.timestamp_col, self.demand_met_col]:
            if col not in df.columns:
                raise ValueError(f"Missing required column '{col}'")

        df[self.timestamp_col] = pd.to_datetime(df[self.timestamp_col], errors="coerce")
        if df[self.timestamp_col].isna().any():
            raise ValueError(f"Column '{self.timestamp_col}' contains non-parseable datetimes")

        # temporal
        df["time_id"] = df[self.timestamp_col].dt.strftime("%H-%M").astype("string")

        # quantitative
        q = self.demand_met_col
        df[f"{q}_sqrd"] = df[q] ** 2
        df[f"log_{q}"] = np.log(df[q] + self.epsilon)
        df[f"log_{q}_sqrd"] = df[f"log_{q}"] ** 2

        if self.co2_col in df.columns:
            df[f"log_{self.co2_col}"] = np.log(df[self.co2_col] + self.epsilon)

        return df

    def get_feature_names_out(self, input_features=None):
        base = []
        base.append("time_id")
        base += [
            f"{self.demand_met_col}_sqrd",
            f"log_{self.demand_met_col}",
            f"log_{self.demand_met_col}_sqrd",
        ]
        # optional; only present if co2 is in input
        base.append(f"log_{self.co2_col}")
        if input_features is not None:
            return np.array(list(input_features) + base)
        return np.array(base)


In [32]:
class DateTimeFeatureAdder(BaseEstimator, TransformerMixin):
    """
    Add datetime-based features from a timestamp column.

    New columns:
      - year (int)
      - month (int)
      - week_of_year (ISO week, int)
      - day (int)
      - hour (int)
      - half_hour (0..47, int)
      - day_of_week (1=Mon..7=Sun, int)
      - is_weekend (0/1, int)


    Parameters
    ----------
    timestamp_col : str, default="timestamp"
        Name of the column containing datetime strings or pd.Timestamp.
    drop_original : bool, default=True
        Whether to drop the original timestamp column after extraction.

    Raises
    ------
    TypeError
        If `timestamp_col` is not found in the DataFrame.
    KeyError
        If `timestamp_col` is not present in X.
\
    """
    def __init__(
        self,
        timestamp_col: str = "timestamp",
        drop_original: bool = False,
    ):
        """
        Initialize the feature adder.

        Parameters
        ----------
        timestamp_col : str
            Column name to parse as datetime.
        """
        if not isinstance(timestamp_col, str):
            raise TypeError("timestamp_col must be a string.")
        if not isinstance(drop_original, bool):
            raise TypeError("drop_original must be a bool.")
        self.timestamp_col = timestamp_col
        self.drop_original = drop_original


    def fit(self, X, y=None):
        """
        No-op fit. Exists for sklearn compatibility.

        Returns
        -------
        self : DateTimeFeatureAdder
        """
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input X must be a pandas DataFrame.")
        if self.timestamp_col not in X.columns:
            raise KeyError(f"Column '{self.timestamp_col}' not found in DataFrame.")

        self.n_features_in_ = X.shape[1]
        self.is_fitted_ = True
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        """
        Transform X by adding:

        - year (int)
        - month (int)
        - week_of_year (int)
        - day (int)
        - hour (int)
        - half_hour (int, 0-47)
        - day_of_week (int, 1=Mon)
        - is_weekend (0/1)

        Parameters
        ----------
        X : pd.DataFrame
            Input DataFrame with a column named `self.timestamp_col`.

        Returns
        -------
        X_out : pd.DataFrame
            Copy of X with the above new columns appended.

        Raises
        ------
        KeyError
            If `self.timestamp_col` is not present in X.
        """
        df = X.copy()
        # Attempt to convert the timestamp column to datetime (if not already)
        try:
            df[self.timestamp_col] = pd.to_datetime(df[self.timestamp_col], errors='raise')
        except Exception as e:
            raise TypeError(f"Column '{self.timestamp_col}' could not be converted to datetime: {e}")

        dt = df[self.timestamp_col]
        df["year"] = dt.dt.year.astype('int32')
        df["month"] = dt.dt.month.astype('int32')
        df["week_of_year"] = dt.dt.isocalendar().week.astype('int32')
        df["day"] = dt.dt.day.astype('int32')
        df["hour"] = dt.dt.hour.astype('int32')
        df["half_hour"]    = (dt.dt.hour * 2 + (dt.dt.minute // 30)).astype("int32")
        df["day_of_week"] = (dt.dt.dayofweek).astype('int32') + 1  # Monday=1
        df["is_weekend"] = (df["day_of_week"] >= 6).astype('int32')

        if self.drop_original:
            df = df.drop(columns=[self.timestamp_col])

        return df

    def get_feature_names_out(self, input_features=None):
        """
        Get the names of the output features.

        Parameters
        ----------
        input_features : array-like, optional
            The input feature names. If None, the original feature names are used.

        Returns
        -------
        np.ndarray
            The output feature names.
        """
        added = ["year","month","week_of_year","day","hour","half_hour","day_of_week","is_weekend"]
        if self.drop_original or input_features is None:
            base = [] if input_features is None else [c for c in input_features if c != self.timestamp_col]
        else:
            base = list(input_features)
        return np.array(base + added, dtype=object)

In [33]:
class GenerationShareAdder(BaseEstimator, TransformerMixin):
    """
    Add percentage‐share features for specified generation columns relative to a total.

    Parameters
    ----------
    generation_cols : List[str]
        Columns whose shares of `total_col` are computed.
    total_col : str, default="total_generation"
        Denominator column.
    suffix : str, default="_share"
        Suffix appended to new share columns.
    as_percent : bool, default=True
        If True, multiply shares by 100; otherwise keep as 0..1 fraction.
    clip_0_100 : bool, default=False
        If True and `as_percent=True`, clip results into [0, 100].
        If True and `as_percent=False`, clip into [0, 1].

    Raises
    ------
    TypeError
        Bad argument types.
    KeyError
        Missing `generation_cols` or `total_col`.
    """

    def __init__(
        self,
        generation_cols: List[str],
        total_col: str = "total_generation",
        suffix: str = "_share",
        as_percent: bool = True,
        clip_0_100: bool = False,
    ):
        """
        Initialize the share adder.

        Parameters
        ----------
        generation_cols : List[str]
            Columns to convert into percentage shares.
        total_col : str
            Column used as the denominator in share calculation.
        suffix : str
            Suffix for the new share columns.

        Raises
        ------
        TypeError
            If `generation_cols` is not a list of strings, or if `total_col` or `suffix` are not strings.
        """
        if not isinstance(generation_cols, list) or not all(isinstance(col, str) for col in generation_cols):
            raise TypeError("generation_cols must be a list of strings.")
        if not isinstance(total_col, str):
            raise TypeError("total_col must be a string.")
        if not isinstance(suffix, str):
            raise TypeError("suffix must be a string.")
        if not isinstance(as_percent, bool):
            raise TypeError("as_percent must be a bool.")
        if not isinstance(clip_0_100, bool):
            raise TypeError("clip_0_100 must be a bool.")

        self.generation_cols = generation_cols
        self.total_col = total_col
        self.suffix = suffix
        self.as_percent = as_percent
        self.clip_0_100 = clip_0_100

    def fit(self, X, y=None):
        """
        No‐op fit for compatibility with sklearn’s transformer API.

        Parameters
        ----------
        X : pd.DataFrame
            Input DataFrame.
        y : Ignored

        Returns
        -------
        self : GenerationShareAdder

        Raises
        ------
        TypeError
            If `X` is not a pandas DataFrame.
        KeyError
            If any of the specified `generation_cols` or `total_col` is not present in the DataFrame.
        """
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input X must be a pandas DataFrame.")
        missing_cols = [col for col in self.generation_cols if col not in X.columns]
        if missing_cols:
            raise KeyError(f"Generation columns {missing_cols} not found in input DataFrame.")
        if self.total_col not in X.columns:
            raise KeyError(f"Total column '{self.total_col}' not found in input DataFrame.")
        return self


    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        """
        Compute and append share columns.

        For each `col` in `generation_cols`, creates a new column
        `col + suffix` = 100 * (X[col] / X[total_col]). Zeros in `total_col`
        are treated as NaN to avoid division‐by‐zero.

        Parameters
        ----------
        X : pd.DataFrame
            Input DataFrame containing `generation_cols` and `total_col`.

        Returns
        -------
        X_out : pd.DataFrame
            Copy of X with additional `<col><suffix>` columns.

        """
        df = X.copy()
        # avoid integer division & div-by-zero
        total = df[self.total_col].astype("float64").replace({0.0: np.nan})
        scale = 100.0 if self.as_percent else 1.0

        for col in self.generation_cols:
            share_col = f"{col}{self.suffix}"
            df[share_col] = (df[col].astype("float64") / total) * scale
            if self.clip_0_100:
                lo, hi = (0.0, 100.0) if self.as_percent else (0.0, 1.0)
                df[share_col] = df[share_col].clip(lower=lo, upper=hi)

        return df


    def get_feature_names_out(self, input_features=None):
        added = [f"{c}{self.suffix}" for c in self.generation_cols]
        base = [] if input_features is None else list(input_features)
        return np.array(base + added, dtype=object)


#### Multi-Quantile Binner

In [34]:
class MultiQuantileBinner(BaseEstimator, TransformerMixin):
    """
    Quantile bin multiple variables, then combine their per-variable bin IDs into
    a single mixed-radix group ID (1-based).

    Example: with bin_specs={'v1':5, 'v2':4}:
      - Fit stores quantile edges for each var.
      - Transform assigns v1_group∈{1..5}, v2_group∈{1..4},
        then builds group_col_name = 1 + (v1_group-1)*4 + (v2_group-1)*1.
    """

    def __init__(
        self,
        bin_specs: dict[str, int],
        group_col_name: str = "quantile_group_id",
        retain_flags: bool = True,
        oob_policy: str = "clip",
        max_oob_rate: float | None = None,
    ):
        """
        Parameters
        ----------
        bin_specs : dict[str, int]
            Mapping of variable -> # of quantile bins (positive integers).
        group_col_name : str, default "quantile_group_id"
            Output column for the combined mixed-radix group ID (1-based).
        retain_flags : bool, default True
            If True, keep per-variable `<var>_group` columns.
        oob_policy : {"clip","edge","error"}, default "clip"
            Handling for values falling outside learned edges at transform time:
              - "clip": send to nearest bin (1 or max)
              - "edge": send to the first bin
              - "error": raise ValueError
        max_oob_rate : float or None, default None
            If set, raise an error when an individual variable sees
            OOB rate > max_oob_rate during transform.
        """
        if not isinstance(bin_specs, dict) or not bin_specs:
            raise ValueError("bin_specs must be a non-empty dict")
        if oob_policy not in {"clip", "edge", "error"}:
            raise ValueError("oob_policy must be one of {'clip','edge','error'}")

        self.bin_specs = self.validate_and_convert_bins(bin_specs)
        self.group_col_name = str(group_col_name)
        self.retain_flags = bool(retain_flags)
        self.oob_policy = oob_policy
        self.max_oob_rate = max_oob_rate

        self.variables_: list[str] | None = None
        self.quantile_edges_: dict[str, list[float]] = {}
        self.bin_sizes_: dict[str, int] = {}
        self.multipliers_: list[int] | None = None
        self.oob_counts_: dict[str, int] = {}

    def fit(self, X: pd.DataFrame, y=None):
        """
        Learn quantile edges for each variable.

        Parameters
        ----------
        X : pd.DataFrame
            Must contain all variables in `bin_specs`.
        """
        self.variables_ = list(self.bin_specs.keys())
        self.quantile_edges_.clear()
        self.bin_sizes_.clear()
        self.oob_counts_.clear()

        eps = 1e-4
        for var in self.variables_:
            n_bins = self.bin_specs[var]
            if var not in X.columns:
                raise ValueError(f"Column '{var}' not found in X")
            qs = np.linspace(0, 1, n_bins + 1)
            raw = X[var].quantile(qs, interpolation="midpoint").values
            vmin, vmax = X[var].min(), X[var].max()
            edges = np.unique(np.concatenate([[vmin - eps], raw, [vmax + eps]]))
            edges.sort()
            self.quantile_edges_[var] = edges.tolist()
            self.bin_sizes_[var] = len(edges) - 1

        bases = [self.bin_sizes_[v] for v in self.variables_]
        m = [1]
        for b in reversed(bases[1:]):
            m.insert(0, m[0] * b)
        self.multipliers_ = m
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        """
        Assign per-variable quantile bins and the combined group ID.

        Returns
        -------
        pd.DataFrame
            X plus `<var>_group` (optional) and `group_col_name`.
        """
        if not self.quantile_edges_:
            raise RuntimeError("Must fit binner before transform()")
        df = X.copy()
        self.oob_counts_ = {var: 0 for var in self.variables_}

        for var in self.variables_:
            edges = self.quantile_edges_[var]
            n = len(edges) - 1
            s = pd.cut(df[var], bins=edges, labels=range(1, n + 1), include_lowest=True, right=True)

            if s.isna().any():
                n_oob = int(s.isna().sum())
                self.oob_counts_[var] += n_oob
                if self.oob_policy == "error":
                    bad = df.loc[s.isna(), var].unique()
                    raise ValueError(f"OOB values for '{var}': {bad[:10]} ...")
                elif self.oob_policy == "clip":
                    below = df[var] < edges[1]
                    s = s.astype("Float64")
                    s.loc[s.isna() & below] = 1
                    s.loc[s.isna() & ~below] = n
                    s = s.astype("Int64")
                else:  # "edge"
                    s = s.fillna(1)

            df[f"{var}_group"] = s.astype(int)

        total = len(df)
        if self.max_oob_rate is not None and total > 0:
            for var, cnt in self.oob_counts_.items():
                rate = cnt / total
                if rate > self.max_oob_rate:
                    raise ValueError(
                        f"OOB rate {rate:.2%} exceeds max_oob_rate={self.max_oob_rate:.2%} for '{var}'"
                    )

        df[self.group_col_name] = 1
        for v, m in zip(self.variables_, self.multipliers_):
            df[self.group_col_name] += (df[f"{v}_group"] - 1) * m

        if not self.retain_flags:
            df.drop(columns=[f"{v}_group" for v in self.variables_], inplace=True)

        return df

    @staticmethod
    def validate_and_convert_bins(bin_specs: dict) -> dict[str, int]:
        converted: dict[str, int] = {}
        for k, v in bin_specs.items():
            try:
                v_int = int(float(v))
                if v_int != float(v) or v_int <= 0:
                    raise ValueError
                converted[str(k)] = v_int
            except (ValueError, TypeError) as e:
                raise TypeError(f"Bin spec '{k}' value '{v}' must be a positive integer") from e
        return converted

    def get_feature_names_out(self, input_features=None):
        names = []
        if self.retain_flags and self.variables_:
            names += [f"{v}_group" for v in self.variables_]
        names.append(self.group_col_name)
        if input_features is not None:
            return np.array(list(input_features) + names)
        return np.array(names)


#### Multi-Median Binner

In [35]:
class MultiMedianBinner(BaseEstimator, TransformerMixin):
    """
    Median-split each variable and combine flags into a 1-based group ID.
    """

    def __init__(self, variables: list[str], group_col_name: str = "median_group_id", retain_flags: bool = True):
        if not isinstance(variables, list) or len(variables) == 0:
            raise ValueError("`variables` must be a non-empty list of column names.")
        if any(not isinstance(v, str) for v in variables):
            raise TypeError("All entries in `variables` must be strings.")
        if not isinstance(group_col_name, str) or not group_col_name:
            raise TypeError("`group_col_name` must be a non-empty string.")
        if not isinstance(retain_flags, bool):
            raise TypeError("`retain_flags` must be a boolean value.")

        self.variables = variables
        self.group_col_name = group_col_name
        self.retain_flags = retain_flags
        self.medians_: dict[str, float] = {}

    def fit(self, X, y=None):
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input X must be a pandas DataFrame.")
        missing = [v for v in self.variables if v not in X.columns]
        if missing:
            raise ValueError(f"Columns not found in input DataFrame: {missing}")
        self.medians_ = X[self.variables].median(skipna=True).to_dict()
        return self

    def transform(self, X):
        """
        Returns
        -------
        pd.DataFrame
            Copy of X with optional `<var>_group` flags (0/1) and `group_col_name`.
        """
        if not self.medians_:
            raise RuntimeError("Must call fit() before transform().")
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input X must be a pandas DataFrame.")
        missing = [v for v in self.variables if v not in X.columns]
        if missing:
            raise ValueError(f"Columns missing at transform time: {missing}")

        df = X.copy()
        # compare each column to its scalar median (aligned by column name)
        flags = (df[self.variables] > pd.Series(self.medians_)).astype(int)

        multipliers = 2 ** np.arange(len(self.variables))[::-1]
        df[self.group_col_name] = flags.values.dot(multipliers) + 1

        if self.retain_flags:
            for var in self.variables:
                df[f"{var}_group"] = flags[var]

        return df

    def get_feature_names_out(self, input_features=None):
        names = []
        if self.retain_flags:
            names += [f"{v}_group" for v in self.variables]
        names.append(self.group_col_name)
        if input_features is not None:
            return np.array(list(input_features) + names)
        return np.array(names)


#### GroupwiseRegressor

In [36]:
class GroupwiseRegressor(BaseEstimator, TransformerMixin):
    """
    Runs separate OLS regressions in each group and computes marginal emission factors.

    For each group k, we fit:
        y_t = α₁ₖ · x₁_t + α₂ₖ · x₂_t + Σ β_i·C(f_i)_t + ε_t
    and compute the marginal effect:
        ME_t = ∂y_t/∂x₁_t = α₁ₖ + 2·α₂ₖ·x₁_t.

    Parameters
    ----------
    y_var : str
        Target column name (e.g. 'tons_co2').
    x_vars : List[str]
        Predictor columns; first is Q, second is Q².
    fe_vars : List[str], optional
        Categorical fixed-effect columns.
    group_col : str
        Column with integer group IDs.
    min_group_size : int
        Minimum observations per group to run regression.
    track_metrics : bool
        If True, store per-group models and metrics.
    verbose : bool
        If True, log progress and metrics.

    Attributes
    ----------
    group_models_ : dict
        Fitted statsmodels results per group (if track_metrics=True).
    group_metrics_ : dict
        Computed metrics per group (if track_metrics=True).
    """
    def __init__(
        self,
        y_var: str = "tons_co2",
        x_vars: List[str] = ["total_generation", "total_generation_sqrd"],
        fe_vars: Optional[List[str]] = None,
        group_col: str = "k",
        min_group_size: int = 10,
        track_metrics: bool = True,
        verbose: bool = True,
        random_state: int | None = 12,
    ):
        if not isinstance(y_var, str):
            raise TypeError("y_var must be a string")
        if not isinstance(x_vars, list) or not x_vars or not all(isinstance(v, str) for v in x_vars):
            raise TypeError("x_vars must be a non-empty list of strings")
        if fe_vars is not None and (not isinstance(fe_vars, list) or not all(isinstance(v, str) for v in fe_vars)):
            raise TypeError("fe_vars must be a list of strings or None")
        if not isinstance(group_col, str):
            raise TypeError("group_col must be a string")
        if not isinstance(min_group_size, int) or min_group_size < 1:
            raise ValueError("min_group_size must be a positive integer")
        if not isinstance(track_metrics, bool):
            raise TypeError("track_metrics must be a boolean")
        if not isinstance(verbose, bool):
            raise TypeError("verbose must be a boolean")

        self.y_var = y_var
        self.x_vars = x_vars
        self.fe_vars = fe_vars or []
        self.group_col = group_col
        self.min_group_size = min_group_size
        self.track_metrics = track_metrics
        self.verbose = verbose
        self.random_state = random_state
        if self.track_metrics:
            self.group_models_: dict[Any, Any] = {}
            self.group_metrics_: dict[Any, dict[str, float]] = {}

    def fit(self, X, y=None):
        if self.random_state is not None:
            np.random.seed(self.random_state)
        if not isinstance(X, pd.DataFrame):
            raise TypeError("X must be a pandas DataFrame")
        if y is None:
            raise ValueError("y must be provided for fitting")
        if len(X) != len(y):
            raise ValueError(f"X and y have different lengths: {len(X)} != {len(y)}")

        df = X.copy()
        df[self.y_var] = np.asarray(y).reshape(-1)

        # avoid uint in formula design matrix
        uint_cols = [c for c in df.columns if str(df[c].dtype).startswith(("uint", "UInt"))]
        if uint_cols:
            df[uint_cols] = df[uint_cols].astype("int64")

        if self.track_metrics:
            self.group_models_.clear()
            self.group_metrics_.clear()

        # cast FEs to ordered categoricals
        if "month" in self.fe_vars:
            df["month"] = pd.Categorical(df["month"].astype(int), categories=range(1, 13), ordered=True)
        if "hour" in self.fe_vars:
            df["hour"] = pd.Categorical(df["hour"].astype(int), categories=range(24), ordered=True)
        if "day_of_week" in self.fe_vars:
            df["day_of_week"] = pd.Categorical(df["day_of_week"].astype(int), categories=range(1, 8), ordered=True)
        if "week_of_year" in self.fe_vars:
            df["week_of_year"] = pd.Categorical(df["week_of_year"].astype(int), categories=range(1, 54), ordered=True)
        if "half_hour" in self.fe_vars:
            df["half_hour"] = pd.Categorical(df["half_hour"].astype(int), categories=range(0, 48), ordered=True)

        self._fitted_groups: list[Any] = []

        for grp, df_grp in df.groupby(self.group_col, sort=True):
            n = len(df_grp)
            if n < self.min_group_size:
                if self.verbose:
                    logging.warning(f"Skipping group {grp!r}: only {n} < {self.min_group_size}")
                continue

            reg = " + ".join(self.x_vars)
            fe = " + ".join(f"C({f})" for f in self.fe_vars)
            formula = f"{self.y_var} ~ {reg}" + (f" + {fe}" if fe else "")

            model = smf.ols(formula, data=df_grp).fit()
            self._fitted_groups.append(grp)

            if self.track_metrics:
                preds = model.predict(df_grp)
                rmse = float(np.sqrt(np.mean((preds - df_grp[self.y_var]) ** 2)))
                mae = float(np.mean(np.abs(preds - df_grp[self.y_var])))
                mape = float(mean_absolute_percentage_error(df_grp[self.y_var], preds))
                self.group_models_[grp] = model
                self.group_metrics_[grp] = {
                    "r2": float(model.rsquared),
                    "rmse": rmse,
                    "mae": mae,
                    "mape": mape,
                    "n_obs": int(n),
                }

        if not self._fitted_groups:
            raise ValueError("No valid groups found for fitting.")
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        """
        Apply groupwise OLS and compute marginal effects ME_t.

        Parameters
        ----------
        X : pd.DataFrame
            Must contain y_var, x_vars, fe_vars, and group_col.

        Returns
        -------
        pd.DataFrame
            Original rows plus 'alpha1', 'alpha2', and 'ME'.

        Raises
        ------
        TypeError
            If X is not a pandas DataFrame.
        ValueError
            If required columns missing or no group qualifies.
        """
        if not getattr(self, "group_models_", None):
            raise RuntimeError("GroupwiseRegressor must be fit before transform/predict.")
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input X must be a pandas DataFrame")

        df = X.copy()

        # keep FE category casting consistent with fit
        if "month" in self.fe_vars:
            df["month"] = pd.Categorical(df["month"].astype(int), categories=range(1, 13), ordered=True)
        if "hour" in self.fe_vars:
            df["hour"] = pd.Categorical(df["hour"].astype(int), categories=range(24), ordered=True)
        if "day_of_week" in self.fe_vars:
            df["day_of_week"] = pd.Categorical(df["day_of_week"].astype(int), categories=range(1, 8), ordered=True)
        if "week_of_year" in self.fe_vars:
            df["week_of_year"] = pd.Categorical(df["week_of_year"].astype(int), categories=range(1, 54), ordered=True)
        if "half_hour" in self.fe_vars:
            df["half_hour"] = pd.Categorical(df["half_hour"].astype(int), categories=range(0, 48), ordered=True)

        df["alpha1"] = np.nan
        df["alpha2"] = np.nan
        df["ME"] = np.nan

        for grp, df_grp in df.groupby(self.group_col, sort=True):
            model = self.group_models_.get(grp)
            if model is None:
                continue
            a1 = model.params.get(self.x_vars[0], np.nan)
            a2 = model.params.get(self.x_vars[1], 0.0)
            idx = df_grp.index

            df.loc[idx, "alpha1"] = a1
            df.loc[idx, "alpha2"] = a2
            df.loc[idx, "ME"] = a1 + 2.0 * a2 * df_grp[self.x_vars[0]]

        return df


    def predict(self, X: pd.DataFrame, predict_type: str = "ME") -> pd.Series:
        """
        Predict marginal effects (default) or CO2 for each row in X using fitted group models.

        Parameters
        ----------
        X : pd.DataFrame
            Must contain x_vars, fe_vars, and group_col.
        predict_type : {"ME","y"}, default "ME"
            "ME": return α1 + 2*α2*Q
            "y" : return model.predict(...) (CO2)

        Returns
        -------
        pd.Series
            Predictions aligned to X.index.
        """

        if not getattr(self, "group_models_", None):
            raise RuntimeError("GroupwiseRegressor must be fit before predict().")
        if not isinstance(X, pd.DataFrame):
            raise TypeError("X must be a pandas DataFrame")

        required = self.x_vars + self.fe_vars + [self.group_col]
        missing = [c for c in required if c not in X.columns]
        if missing:
            raise ValueError(f"Missing columns in input DataFrame: {missing}")

        df = X.copy()
        # consistent FE casting
        if "month" in self.fe_vars:
            df["month"] = pd.Categorical(df["month"].astype(int), categories=range(1, 13), ordered=True)
        if "hour" in self.fe_vars:
            df["hour"] = pd.Categorical(df["hour"].astype(int), categories=range(24), ordered=True)
        if "day_of_week" in self.fe_vars:
            df["day_of_week"] = pd.Categorical(df["day_of_week"].astype(int), categories=range(1, 8), ordered=True)
        if "week_of_year" in self.fe_vars:
            df["week_of_year"] = pd.Categorical(df["week_of_year"].astype(int), categories=range(1, 54), ordered=True)
        if "half_hour" in self.fe_vars:
            df["half_hour"] = pd.Categorical(df["half_hour"].astype(int), categories=range(0, 48), ordered=True)

        out = pd.Series(index=df.index, dtype=float)

        for grp, df_grp in df.groupby(self.group_col, sort=True):
            model = self.group_models_.get(grp)
            if model is None:
                continue

            if predict_type == "y":
                preds = model.predict(df_grp)
            else:
                a1 = model.params.get(self.x_vars[0], np.nan)
                a2 = model.params.get(self.x_vars[1], 0.0)
                Q = df_grp[self.x_vars[0]]
                preds = a1 + 2.0 * a2 * Q

            out.loc[df_grp.index] = preds

        return out

    def get_metrics(self, summarise: bool = True) -> Union[dict, pd.DataFrame]:
        """
        Get the metrics for each group.

        Parameters
        ----------
        summarise : bool, default=True
            If True, return a summary DataFrame; otherwise return raw metrics dict.

        Returns
        -------
        dict or pd.DataFrame
            If summarise=True, returns a DataFrame with group metrics.
            If False, returns the raw metrics dictionary.

        Raises
        ------
        RuntimeError
            If track_metrics was not set to True during initialization.
        """
        if not self.track_metrics:
            raise RuntimeError("Metrics tracking is disabled. Set track_metrics=True to enable.")
        if summarise:
            df = pd.DataFrame.from_dict(self.group_metrics_, orient="index")
            df.index.name = self.group_col
            df.reset_index(inplace=True)
            return df
        return self.group_metrics_

### Running Models

#### Utilities

In [37]:
def _apply_fitted_preprocessing(user_pipeline: Pipeline, X: pd.DataFrame) -> pd.DataFrame:
    """
    Apply all *already-fitted* steps in a pipeline except the final estimator,
    without constructing a new sklearn Pipeline (avoids 'Pipeline not fitted' warnings).

    Parameters
    ----------
    user_pipeline : Pipeline
        A pipeline that has already been fitted (on train) and whose final step
        is the estimator (e.g., GroupwiseRegressor).
    X : pd.DataFrame
        Raw features to transform through the fitted preprocessing steps.

    Returns
    -------
    pd.DataFrame
        The transformed features as a DataFrame. If a transformer returns a numpy array,
        we try to retrieve column names via `get_feature_names_out()`; otherwise we fall
        back to the original column names.
    """
    Z = X
    last_transformer = None

    for _, step in user_pipeline.steps[:-1]:
        if hasattr(step, "transform"):
            Z = step.transform(Z)
            last_transformer = step

    if isinstance(Z, pd.DataFrame):
        return Z

    # Try to recover column names
    cols = None
    try:
        cols = user_pipeline[:-1].get_feature_names_out()  # type: ignore[index]
    except Exception:
        try:
            if last_transformer is not None and hasattr(last_transformer, "get_feature_names_out"):
                cols = last_transformer.get_feature_names_out()  # type: ignore[assignment]
        except Exception:
            cols = None

    if cols is None:
        cols = X.columns
    return pd.DataFrame(Z, index=X.index, columns=list(cols))


In [38]:
def compute_me_for_split(
    fitted_pipeline: Pipeline,
    X: pd.DataFrame,
    split_name: str | None = None,
    id_cols: list[str] = ("timestamp", "city"),
    include_params: bool = True,
    keep_cols: list[str] = ("demand_met", "tons_co2"),
) -> pd.DataFrame:
    """
    Use a FITTED pipeline to compute marginal emissions (ME) for a single features DataFrame.

    Parameters
    ----------
    fitted_pipeline : Pipeline
        A pipeline that has already been fit on the training data. Its final step must be
        GroupwiseRegressor, whose transform adds 'ME' (and 'alpha1','alpha2').
    X : pd.DataFrame
        Feature table to transform. Must include the columns required by the pipeline’s
        feature steps and binner (e.g., weather vars), plus any IDs you want to keep.
    split_name : str, optional
        If provided, a 'split' column is added with this value ('train'/'validation'/'test'/etc).
    id_cols : list[str], default ('timestamp','city')
        Identifier columns to carry into the output if present in `X` after transform.
    include_params : bool, default True
        If True, also include 'alpha1' and 'alpha2' in the output.
    keep_cols : list[str], default ('demand_met','tons_co2')
        Additional columns to include if present (useful for diagnostics).

    Returns
    -------
    pd.DataFrame
        One row per input row with at least: id_cols ∩ columns, 'ME', and optionally
        'alpha1','alpha2', the regressor’s group column, keep_cols, and 'split'.
    """
    # Transform through all steps → last step (GroupwiseRegressor) computes ME
    out = fitted_pipeline.transform(X)

    # Final estimator for group column name
    reg = getattr(fitted_pipeline, "_final_estimator", None)
    gcol = getattr(reg, "group_col", None)

    # Build column list in a safe, present-only way
    cols: list[str] = [c for c in id_cols if c in out.columns]
    if "ME" not in out.columns:
        raise RuntimeError("Pipeline transform did not produce 'ME'. Was the final estimator fitted?")
    cols.append("ME")

    if include_params:
        for c in ("alpha1", "alpha2"):
            if c in out.columns:
                cols.append(c)

    if gcol and gcol in out.columns:
        cols.append(gcol)

    for c in keep_cols:
        if c in out.columns and c not in cols:
            cols.append(c)

    result = out[cols].copy()
    if split_name is not None:
        result["split"] = split_name
    return result

In [39]:
def evaluate_on_split(
        regression_model: GroupwiseRegressor,
        full_df: pd.DataFrame
) -> pd.DataFrame:
    """
    After pipeline.transform → full_df with group IDs & original y_var,
    compute per‑group r2/rmse/mae/n_obs using reg.group_models_.

    Parameters
    ----------
    reg : GroupwiseRegressor
        Fitted GroupwiseRegressor instance with group_models_ populated.
    full_df : pd.DataFrame
        DataFrame containing the original y_var and group_col.

    Returns
    -------
    pd.DataFrame
        DataFrame with group metrics: r2, rmse, mae, n_obs.
    """
    df = full_df.copy()
    gcol = regression_model.group_col
    yname = regression_model.y_var

    if gcol not in df.columns or yname not in df.columns:
        missing = [c for c in (gcol, yname) if c not in df.columns]
        raise KeyError(f"Required columns missing: {missing}")

    # Use the regressor's predict to ensure FE category handling is consistent
    y_true = df[yname]
    y_pred = regression_model.predict(df, predict_type="y")

    rows = []
    for grp, idx in df.groupby(gcol).groups.items():
        yt = y_true.loc[idx]
        yp = y_pred.loc[idx].dropna()
        # align just in case
        yt = yt.loc[yp.index]
        if len(yt) == 0:
            continue
        rows.append({
            "group": grp,
            "r2": r2_score(yt, yp),
            "rmse": root_mean_squared_error(yt, yp),
            "mae": mean_absolute_error(yt, yp),
            "mape": mean_absolute_percentage_error(yt, yp),
            "n_obs": int(len(yt)),
        })

    mdf = pd.DataFrame(rows)
    if mdf.empty:
        # return empty with expected columns
        mdf = pd.DataFrame(columns=["group","r2","rmse","mae","mape","n_obs"])
    return mdf

In [40]:
def fit_and_export_marginal_emissions(
    pipeline: Pipeline,
    x_splits: dict,
    y_splits: dict,
    out_parquet_path: str,
    *,
    id_cols: list[str] = ("timestamp", "city"),
    include_params: bool = True,
    keep_cols: list[str] = ("demand_met", "tons_co2"),
    order_splits: list[str] = ("train", "validation", "test"),
    save_mode: str = "single",              # "single" | "per_split"
    compression: str | None = "snappy",     # passed to pandas.to_parquet
    return_df: bool = True,                 # set False on huge runs
) -> pd.DataFrame:
    """
    Fit the pipeline on the train split, compute marginal emissions (ME) for each split,
    concatenate, and save to a single Parquet file.

    Parameters
    ----------
    pipeline : Pipeline
        Your full pipeline: [FeatureAddition → Binner → GroupwiseRegressor].
    x_splits : dict
        Feature splits, e.g. {"train": X_train, "validation": X_val, "test": X_test}.
    y_splits : dict
        Target splits, e.g. {"train": y_train, "validation": y_val, "test": y_test}.
        Only the train target is used for fitting; others are not needed for transform.
    out_parquet_path : str
        File path for the output Parquet dataset.
    id_cols : list[str], default ('timestamp','city')
        Identifier columns to include if present.
    include_params : bool, default True
        Include 'alpha1' and 'alpha2' in the export.
    keep_cols : list[str], default ('demand_met','tons_co2')
        Additional useful columns to include if present.
    order_splits : list[str], default ('train','validation','test')
        Order in which to compute and stack splits.
    save_mode : {"single","per_split"}, default "single"
        Use "per_split" on HPC/MPI (let rank 0 write or give each rank a different path).
    compression : str or None, default "snappy"
        Parquet compression codec (requires pyarrow/fastparquet support).
    return_df : bool, default True
        If False, skip building the concatenated DataFrame in memory.


    Returns
    -------
    pd.DataFrame or None
        Concatenated results if return_df=True and save_mode="single"; otherwise None.


    Notes
    -----
    - Binner quantile edges and groupwise OLS coefficients are learned on TRAIN only.
    - Validation and test are transformed using those learned edges/coefficients.
    - In MPI jobs, prefer save_mode="per_split" or call this only on rank 0.
    - Binner edges and group OLS coefs are learned on train only.
    """
    out_parquet_path = Path(out_parquet_path)

    # Fit on train
    X_tr = x_splits["train"]
    y_tr = y_splits["train"]
    _ = pipeline.fit_transform(X_tr, y_tr)

    if save_mode not in {"single", "per_split"}:
        raise ValueError("save_mode must be 'single' or 'per_split'.")

    # Compute ME for each requested split
    parts: list[pd.DataFrame] = []
    for split in order_splits:
        if split not in x_splits:
            continue
        df_me = compute_me_for_split(
            fitted_pipeline=pipeline,
            X=x_splits[split],
            split_name=split,
            id_cols=id_cols,
            include_params=include_params,
            keep_cols=keep_cols,
        )

        if save_mode == "per_split":
            split_path = out_parquet_path.with_name(
                f"{out_parquet_path.stem}__{split}{out_parquet_path.suffix or '.parquet'}"
            )
            split_path.parent.mkdir(parents=True, exist_ok=True)
            df_me.to_parquet(split_path, index=False, compression=compression)
            # optionally avoid keeping in memory on huge runs
            if return_df:
                parts.append(df_me)
        else:
            parts.append(df_me)

    if save_mode == "single":
        final = pd.concat(parts, ignore_index=True) if parts else pd.DataFrame()
        out_parquet_path.parent.mkdir(parents=True, exist_ok=True)
        final.to_parquet(out_parquet_path, index=False, compression=compression)
        print(f"[SAVE] Wrote marginal emissions to {out_parquet_path} (rows={len(final):,})")
        return final if return_df else None
    else:
        print(f"[SAVE] Wrote per-split Parquet files next to {out_parquet_path}")
        if return_df:
            return pd.concat(parts, ignore_index=True) if parts else pd.DataFrame()
        return None

#### Runners & Orchestrators

In [41]:
def run_regressor_model(
    user_pipeline: Pipeline,
    x_df: pd.DataFrame,
    y_df: pd.Series | pd.DataFrame,
    split_name: str,
    extra_info: dict | None = None,
    return_model: bool = False,
    random_state: int = 12,
    interval_hours: float = 0.5,
    *,
 model_id_hash: str | None = None,
    params_json_str: str | None = None,
) -> tuple[pd.DataFrame, list[str], GroupwiseRegressor | dict]:
    """
    Run a pipeline on one split, compute per-group metrics, attach energy weights,
    and compute diagnostics (pooled CO₂ fit + finite-difference ME checks).

    Parameters
    ----------
    user_pipeline : Pipeline
        Full pipeline [FeatureAddition → (Binner) → GroupwiseRegressor].
    x_df : pd.DataFrame
        Features for the split.
    y_df : pd.Series or single-column pd.DataFrame
        Target for the split.
    split_name : {"train","validation","test"}
        Which split to run.
    extra_info : dict, optional
        Extra metadata to stamp onto the output rows.
    return_model : bool, default False
        If True, returns the final estimator as the 3rd tuple item; otherwise returns extras dict.
    random_state : int, default 12
        Random seed for reproducibility.
    interval_hours : float, default 0.5
        Duration represented by each row (half-hourly = 0.5).
    model_id_hash : str, optional
        If provided, stamp this precomputed run-level hash (recommended).
        If None, a local signature is computed (useful for ad-hoc calls).
    params_json_str : str, optional
        Pre-rendered pipeline params JSON to stamp; if None, it is computed.

    Returns
    -------
    metrics_df : pd.DataFrame
        Per-group metrics with added 'energy_MWh' and metadata columns.
    x_cols_used : list[str]
        Regressor feature names used by the GroupwiseRegressor (x_vars + fe_vars).
    model_or_extras : GroupwiseRegressor | dict
        If return_model=True → the fitted final estimator; else a dict of diagnostics.
    """
    np.random.seed(random_state)

    for col in x_df.columns:
        dt = x_df[col].dtype
        if str(dt).startswith(("uint", "UInt")):
            x_df[col] = x_df[col].astype("int64")

    if split_name not in ("train", "validation", "test"):
        raise ValueError(f"split_name must be 'train', 'validation', or 'test' (got {split_name!r})")

    X = x_df.copy()
    if isinstance(y_df, pd.DataFrame):
        if y_df.shape[1] != 1:
            raise ValueError("y_df must be a Series or single-column DataFrame.")
        y_ser = y_df.iloc[:, 0]
    else:
        y_ser = y_df

    # Use provided model_id_hash (from orchestrator) or compute a local one
    if model_id_hash is None:
        model_id_hash, _ = signature_for_run(
            user_pipeline,
            x_columns=list(X.columns),
            y=y_ser,
            random_state=random_state,
            eval_splits=(split_name,),   # local call; orchestrator passes a shared hash
            compute_test=False,
            extra_info=extra_info,
        )

    if params_json_str is None:
        params_json_str = json.dumps(
            user_pipeline.get_params(deep=True),
            sort_keys=True, separators=(",", ":"), default=str
        )

    extras: dict[str, Any] = {}

    if split_name == "train":
        # Fit → metrics from regressor
        _ = user_pipeline.fit_transform(X, y_ser)
        model = user_pipeline._final_estimator  # type: ignore[attr-defined]
        metrics_df = model.get_metrics(summarise=True).reset_index(drop=True)

        # Canonicalize group col to "group"
        if model.group_col in metrics_df.columns:
            metrics_df = metrics_df.rename(columns={model.group_col: "group"})
        elif "group" not in metrics_df.columns:
            metrics_df = metrics_df.rename(columns={metrics_df.columns[0]: "group"})

        # Preprocessed rows for weights & diagnostics
        x_tr = _apply_fitted_preprocessing(user_pipeline, X)
        x_tr[model.y_var] = np.asarray(y_ser, dtype=float)

        # Energy weights
        w = _compute_group_energy_weights(
            df=x_tr, group_col=model.group_col, q_col=model.x_vars[0], interval_hours=interval_hours
        ).rename(columns={model.group_col: "group"})
        metrics_df = metrics_df.merge(w, on="group", how="left")

        # Diagnostics (in-sample)
        extras["pooled_co2"] = pooled_co2_metrics(
            model, x_tr, y_col=model.y_var, group_col=model.group_col
        )
        me_df = model.transform(x_tr)
        fd_df = finite_difference_me_metrics(
            df=me_df,
            time_col="timestamp" if "timestamp" in me_df.columns else "time_id",
            q_col=model.x_vars[0],
            y_col=model.y_var,
            me_col="ME",
            group_keys=[k for k in ("city",) if k in me_df.columns],
        )
        extras["fd_me_by_city"] = fd_df.to_dict(orient="records") if not fd_df.empty else []
        extras["fd_me_pooled"] = (
            fd_df.loc[fd_df["city"] == "ALL"].iloc[0].to_dict()
            if (not fd_df.empty and "city" in fd_df.columns and "ALL" in fd_df["city"].values)
            else (fd_df.sort_values("n_pairs", ascending=False).iloc[0].to_dict() if not fd_df.empty else {})
        )

    else:
        # Use fitted preprocessing + regressor
        model = user_pipeline._final_estimator  # type: ignore[attr-defined]
        x_tr = _apply_fitted_preprocessing(user_pipeline, X)

        if model.group_col not in x_tr.columns:
            raise KeyError(
                f"Group column '{model.group_col}' is missing after transform. "
                "Ensure your binner outputs it."
            )

        x_tr[model.y_var] = np.asarray(y_ser, dtype=float)

        # Per-group metrics
        metrics_df = evaluate_on_split(model, x_tr)

        # Energy weights
        w = _compute_group_energy_weights(
            df=x_tr, group_col=model.group_col, q_col=model.x_vars[0], interval_hours=interval_hours
        ).rename(columns={model.group_col: "group"})
        metrics_df = metrics_df.merge(w, on="group", how="left")

        # Out-of-sample diagnostics
        extras["pooled_co2"] = pooled_co2_metrics(
            model, x_tr, y_col=model.y_var, group_col=model.group_col
        )
        me_df = model.transform(x_tr)
        fd_df = finite_difference_me_metrics(
            df=me_df,
            time_col="timestamp" if "timestamp" in me_df.columns else "time_id",
            q_col=model.x_vars[0],
            y_col=model.y_var,
            me_col="ME",
            group_keys=[k for k in ("city",) if k in me_df.columns],
        )
        extras["fd_me_by_city"] = fd_df.to_dict(orient="records") if not fd_df.empty else []
        extras["fd_me_pooled"] = (
            fd_df.loc[fd_df["city"] == "ALL"].iloc[0].to_dict()
            if (not fd_df.empty and "city" in fd_df.columns and "ALL" in fd_df["city"].values)
            else (fd_df.sort_values("n_pairs", ascending=False).iloc[0].to_dict() if not fd_df.empty else {})
        )

    # Stamp metadata
    metrics_df["data_split"] = split_name
    metrics_df["model_id_hash"] = model_id_hash
    metrics_df["random_state"] = random_state
    metrics_df["pipeline_params_json"] = params_json_str
    metrics_df["log_time"] = datetime.now().isoformat()

    model = user_pipeline._final_estimator  # type: ignore[attr-defined]
    metrics_df["x_columns_used"] = ",".join(model.x_vars + model.fe_vars)
    for k, v in (extra_info or {}).items():
        metrics_df[k] = v

    x_cols_used = model.x_vars + model.fe_vars
    print(f"[LOG] {len(metrics_df)} rows for split={split_name}, model_id={model_id_hash}, random_state={random_state}")

    return (metrics_df, x_cols_used, model) if return_model else (metrics_df, x_cols_used, extras)

In [42]:
def regressor_orchestrator(
        user_pipeline: Pipeline,
        x_splits: dict,
        y_splits: dict,
        log_csv_path: str | None = "marginal_emissions_log.csv",   # legacy
        extra_info: dict | None = None,
        force_run: bool = False,
        force_overwrite: bool = False,
        random_state: int = 12,
        group_col_name: str = "group",
        interval_hours: float = 0.5,
        eval_splits: tuple[str, ...] | None = None,
        compute_test: bool = False,
        # rotating CSV
        results_dir: str | None = None,
        file_prefix: str | None = None,
        max_log_mb: int = 95,
        fsync: bool = True,
) -> pd.DataFrame | None:
    """
    Fit/evaluate a pipeline on train/validation/test, summarise metrics, and append to a CSV log.

    Parameters
    ----------
    user_pipeline : Pipeline
        Full pipeline, typically [FeatureAddition → Binner → GroupwiseRegressor].
    x_splits : dict
        Must include "train" and "validation". Include "test" iff compute_test=True.
    y_splits : dict
        Target splits with the same keys as x_splits.  Must include "train" and "validation". Include "test" iff compute_test=True.
     log_csv_path : str, optional
        Legacy path; used only to infer default results_dir/file_prefix if those are None.
    extra_info : dict, optional
        Extra metadata to stamp onto per-split logs (propagates into `run_regressor_model`).
    force_run : bool, default=False
        If False and an identical model signature was previously logged, skip this run.
    force_overwrite : bool, default=False
        If True, allows re-logging the same model_id_hash (previous rows are NOT removed here;
        use `save_summary_to_csv(..., force_overwrite=True)` for row replacement).
    random_state : int, default=12
        Random seed recorded in the model signature and summary.
    group_col_name : str, default="group"
        Canonical group column name used by `summarise_metrics_logs` for nested metrics.

    Returns
    -------
    pd.DataFrame or None
        One-row summary DataFrame if the run executes; None if skipped due to prior identical log.

    Notes
    -----
    - The model signature (hash) is computed from pipeline parameters, feature columns, target name(s),
      random_state, and any `extra_info`. If unchanged and `force_run=False`, the run is skipped.
    - `x_columns` recorded in the summary are taken from the **train** split’s evaluation result.
    """
    # in regressor_orchestrator before signature_for_run(...)
    if eval_splits is None:
        eval_splits = ("train","validation","test") if compute_test else ("train","validation")
    compute_test = ("test" in eval_splits)  # ← keep hash consistent with actual splits

    # One signature for the whole run (based on TRAIN)
    model_key, sig = signature_for_run(
        user_pipeline,
        x_columns=list(x_splits["train"].columns),
        y=y_splits["train"],
        random_state=random_state,
        eval_splits=eval_splits,
        compute_test=compute_test,
        extra_info=extra_info,
    )

    # Resolve dir + prefix (fallback to legacy path)
    if results_dir is None or file_prefix is None:
        base = Path(log_csv_path or "marginal_emissions_log.csv")
        inferred_dir = base.parent if str(base.parent) != "" else Path(".")
        inferred_prefix = base.stem
        results_dir = results_dir or str(inferred_dir)
        file_prefix = file_prefix or inferred_prefix

    # De-dupe via index
    if not force_run and not force_overwrite:
        if is_model_logged_rotating_csv(model_key, results_dir, file_prefix):
            print(f"[SKIP] Model already logged (hash: {model_key})")
            return None

    # Precompute params JSON once (consistent across splits)
    params_json_str = json.dumps(
        user_pipeline.get_params(deep=True),
        sort_keys=True, separators=(",", ":"), default=str
    )

    logs, pooled_extras, fd_extras = {}, {}, {}
    x_cols_used: list[str] | None = None

    for split in eval_splits:
        metrics_df, x_cols_used, extras = run_regressor_model(
            user_pipeline=user_pipeline,
            x_df=x_splits[split],
            y_df=y_splits[split],
            split_name=split,
            extra_info=extra_info,
            return_model=False,
            random_state=random_state,
            interval_hours=interval_hours,
            model_id_hash=model_key,          # shared ID across splits
            params_json_str=params_json_str,  # shared params JSON
        )
        logs[split] = metrics_df
        pooled_extras[split] = extras.get("pooled_co2", {})
        fd_extras[split] = extras.get("fd_me_pooled", {})

    summary_df = summarise_metrics_logs(
        train_logs=logs["train"],
        val_logs=logs["validation"],
        test_logs=logs.get("test"),
        user_pipeline=user_pipeline,
        x_columns=x_cols_used or [],
        random_state=random_state,
        group_col_name=group_col_name,           # <- use the parameter you accept
        pooled_metrics_by_split=pooled_extras,
        fd_me_metrics_by_split=fd_extras,
    )

    save_summary_to_rotating_csv(
        summary_df,
        results_dir=results_dir,
        file_prefix=file_prefix,
        max_mb=max_log_mb,
        force_overwrite=force_overwrite,
        fsync=fsync,
    )
    return summary_df

#### Grid Search

In [43]:
def run_grid_search(
        base_feature_pipeline: Pipeline,
        regressor_cls,
        regressor_kwargs: dict,
        grid_config: list[dict],
        x_splits: dict,
        y_splits: dict,
        log_path: str | None,  # legacy; optional now
        global_extra_info: dict | None = None,
        force_run: bool = False,
        force_overwrite: bool = False,
        base_feature_pipeline_name: str = "BaseFeaturePipeline",
        eval_splits: tuple[str, ...] = ("train","validation"),
        results_dir: str | None = None,
        file_prefix: str | None = None,
        max_log_mb: int = 95,
        fsync: bool = True,
) -> None:
    """
    Execute a series of [features → binner → regressor] runs and log one summary row per config.

    Parameters
    ----------
    base_feature_pipeline : Pipeline
        Preprocessing steps applied before binning. This object is cloned per run to avoid state leakage.
    regressor_cls : type
        Estimator class to instantiate for the final step (e.g., GroupwiseRegressor).
    regressor_kwargs : dict
        Baseline kwargs for the regressor. Per-config overrides from `grid_config` are merged on top.
        IMPORTANT: This function will not mutate the caller's dict.
    grid_config : list of dict
        Each item should contain:
            - "binner_class": class (e.g., MultiQuantileBinner or MultiMedianBinner)
            - "binner_kwargs": dict of init args for the binner
            - "label": str label for printing/logging (optional)
            - Optional: "x_vars", "fe_vars" to override the regressor’s predictors per-config
            - Optional: anything else you want echoed into `extra_info`
    x_splits, y_splits : dict
        Dicts keyed by {"train","validation","test"} with DataFrames/Series for each split.
    log_path : str
        CSV path where each successful config appends one summary row.
    global_extra_info : dict, optional
        Extra metadata stamped into each run’s logs.
    force_run, force_overwrite : bool
        Passed through to `regressor_orchestrator`.
    base_feature_pipeline_name : str, default "BaseFeaturePipeline"
        Step name used for the features sub-pipeline.

    Returns
    -------
    None
        Prints progress and writes rows to `log_path`. Skips silently (with a message) if a config
        is already logged and `force_run=False`.

    Notes
    -----
    - We clone `base_feature_pipeline` per run to avoid cross-config state sharing.
    - If a binner provides `group_col_name` and the regressor does not specify `group_col`,
      we set the regressor’s `group_col` to match.
    - If a config provides `x_vars`/`fe_vars`, they override the baseline `regressor_kwargs`.
    """
    missing_x = [s for s in eval_splits if s not in x_splits]
    missing_y = [s for s in eval_splits if s not in y_splits]
    if missing_x or missing_y:
        raise KeyError(f"Missing splits: X{missing_x} Y{missing_y}")

    total = len(grid_config)
    for i, raw_config in enumerate(grid_config, start=1):
        config = dict(raw_config)
        binner_class = config["binner_class"]
        binner_kwargs = dict(config.get("binner_kwargs", {}))
        label = config.get("label", binner_class.__name__)

        reg_kwargs = dict(regressor_kwargs)
        if "x_vars" in config:
            reg_kwargs["x_vars"] = list(config["x_vars"])
        if "fe_vars" in config:
            reg_kwargs["fe_vars"] = list(config["fe_vars"])
        reg_kwargs["random_state"] = reg_kwargs.get("random_state", 12)

        binner_group_col = binner_kwargs.get("group_col_name")
        if binner_group_col and "group_col" not in reg_kwargs:
            reg_kwargs["group_col"] = binner_group_col

        try:
            features_step = clone(base_feature_pipeline)
        except Exception:
            features_step = base_feature_pipeline

        binner = binner_class(**binner_kwargs)
        regressor = regressor_cls(**reg_kwargs)

        full_pipeline = Pipeline([
            (base_feature_pipeline_name, features_step),
            (binner_class.__name__, binner),
            (regressor_cls.__name__, regressor),
        ])

        extra_info = {
            "binner_class": binner_class.__name__,
            "binner_params": binner_kwargs,
            "regressor_params": reg_kwargs,
            "grid_label": label,
            **(global_extra_info or {}),
        }

        rank_tag = ""
        try:
            _, rank, size = _mpi_context()
            rank_tag = f"[R{rank}/{max(size-1,0)}] "
        except Exception:
            pass
        print(f"\n{rank_tag}[GRID {i}/{total}] {label}")

        try:
            summary_df = regressor_orchestrator(
                user_pipeline=full_pipeline,
                x_splits=x_splits,
                y_splits=y_splits,
                log_csv_path=log_path,            # legacy OK
                extra_info=extra_info,
                force_run=force_run,
                force_overwrite=force_overwrite,
                random_state=reg_kwargs["random_state"],
                eval_splits=eval_splits,
                # NEW
                results_dir=results_dir,
                file_prefix=file_prefix,
                max_log_mb=max_log_mb,
                fsync=fsync,
                )
            if summary_df is not None:
                print(f"[GRID] Logged: {label}")
            else:
                print(f"[GRID] Skipped (already logged): {label}")
        except Exception as e:
            print(f"[GRID] ERROR in '{label}': {type(e).__name__}: {e}")
            continue

In [44]:
def run_grid_search_auto(
        base_feature_pipeline,
        regressor_cls,
        regressor_kwargs: dict,
        grid_config: list[dict],
        x_splits: dict,
        y_splits: dict,
        *,
        # logging/rotation knobs
        results_dir: str,
        file_prefix: str,
        max_log_mb: int = 95,
        fsync: bool = False,              # set True on HPC if you want durable writes
        # orchestration
        base_feature_pipeline_name: str = "FeatureAdditionPipeline",
        eval_splits: tuple[str, ...] = ("train","validation"),
        force_run: bool = False,
        force_overwrite: bool = False,
        distribute: str = "auto",         # "auto" | "mpi" | "single"
        dist_mode: str = "stride",        # "stride" | "chunked"
        seed: int = 12,
) -> None:
    """
    Single-node or MPI-parallel grid search runner.

    - Auto-detects MPI and splits `grid_config` across ranks.
    - Ensures per-rank deterministic RNG via `seed + rank`.
    - Uses rotating CSV logging with per-file & index locks.

    Parameters are passed straight to `run_grid_search`, except we slice `grid_config`.

    Parameters
    ----------
    base_feature_pipeline: Pipeline
        The base feature pipeline to use for each config.
    regressor_cls: Type[BaseEstimator]
        The regression model class to use.
    regressor_kwargs: dict
        Keyword arguments to pass to the regression model.
    grid_config: list[dict]
        The grid search configuration to use.
    x_splits: dict
        The input feature splits.
    y_splits: dict
        The target variable splits.
    results_dir: str
        The directory to save results.
    file_prefix: str
        The prefix for result files.
    max_log_mb: int
        The maximum log file size in MB.
    naming: PartNaming | None
        Optional naming scheme for output files.
    fsync: bool
        Whether to fsync log files (for durability).
    base_feature_pipeline_name: str
        The name of the base feature pipeline.
    eval_splits: tuple[str, ...]
        The evaluation splits to use.
    force_run: bool
        Whether to force re-running of existing configs.
    force_overwrite: bool
        Whether to force overwriting of existing results.
    distribute: str
        The distribution strategy to use.
    dist_mode: str
        The distribution mode to use.
    seed: int
        The random seed to use.

    Returns
    -------
    None
        Logs the results of the grid search.
    """
    comm, rank, size = _mpi_context()
    if distribute == "auto":
        distribute = "mpi" if size > 1 else "single"

    # Partition the configs
    local_configs = _distribute_configs(grid_config, rank=rank, size=size, mode=dist_mode) \
                    if distribute == "mpi" else grid_config
    if not local_configs:
        if rank == 0:
            print("[GRID] No configs assigned (empty grid or partition).")
        return

    # Per-rank RNG — override/augment existing random_state
    local_reg_kwargs = dict(regressor_kwargs)
    local_reg_kwargs["random_state"] = int(local_reg_kwargs.get("random_state", seed))

    if rank == 0 and distribute == "mpi":
        print(f"[MPI] size={size} → ~{len(grid_config)/max(size,1):.1f} configs per rank")
    else:
        if distribute == "mpi":
            print(f"[MPI] rank={rank}/{size-1} assigned {len(local_configs)} configs")

    run_grid_search(
        base_feature_pipeline=base_feature_pipeline,
        regressor_cls=regressor_cls,
        regressor_kwargs=local_reg_kwargs,
        grid_config=local_configs,
        x_splits=x_splits,
        y_splits=y_splits,
        log_path=None,  # legacy path unused when using rotating logs
        global_extra_info={"runner_rank": rank, "runner_size": size},
        force_run=force_run,
        force_overwrite=force_overwrite,
        base_feature_pipeline_name=base_feature_pipeline_name,
        eval_splits=eval_splits,
        results_dir=results_dir,
        file_prefix=file_prefix,
        max_log_mb=max_log_mb,
        fsync=fsync,
    )

    # Optional barrier for neat logs
    try:
        comm.Barrier()
    except Exception:
        pass
    if rank == 0:
        print("[GRID] Completed (all ranks).")


#### Parameter Grid

In [45]:
def all_nonempty_subsets(columns: list[str]) -> list[list[str]]:
    """All non-empty subsets preserving input order."""
    return [list(c) for i in range(1, len(columns) + 1) for c in combinations(columns, i)]

In [46]:
def get_fe_vars(all_cols: list[str], x_vars: list[str]) -> list[str]:
    """Complement of x_vars within all_cols."""
    xset = set(x_vars)
    return [c for c in all_cols if c not in xset]

In [47]:
def build_x_fe_combinations_disjoint(
    candidate_x_vars: list[str],
    candidate_fe_vars: list[str],
    x_var_length: int = 2,
    max_fe_len: int | None = None,
    *,
    allow_empty_fe: bool = False,
) -> list[dict[str, Any]]:
    """
    Generate all disjoint non-empty combinations of x_vars and fe_vars.

    Parameters
    ----------
    candidate_x_vars : list of str
        Columns eligible to be used as predictors (x_vars).
    candidate_fe_vars : list of str
        Columns eligible to be used as fixed effects (fe_vars).
    x_var_length : int
        Number of x_vars to include in each combination.
    max_fe_len : int | None
        Maximum number of fe_vars to include in each combination.
    allow_empty_fe : bool
        Whether to allow empty fe_vars in the combinations.

    Returns
    -------
    list of dicts
        Each dict has keys: {'x_vars': [...], 'fe_vars': [...]}
    """
    if x_var_length < 1:
        raise ValueError("x_var_length must be >= 1")
    if len(candidate_x_vars) < x_var_length:
        raise ValueError("Not enough candidate_x_vars for requested x_var_length")

    results: list[dict[str, Any]] = []

    x_subsets = [list(c) for c in combinations(candidate_x_vars, x_var_length)]
    fe_pool = [list(c) for i in range(0 if allow_empty_fe else 1, len(candidate_fe_vars) + 1)
               for c in combinations(candidate_fe_vars, i)]

    for x_vars in x_subsets:
        for fe_vars in fe_pool:
            if max_fe_len is not None and len(fe_vars) > max_fe_len:
                continue
            if set(x_vars).isdisjoint(fe_vars):
                results.append({"x_vars": x_vars, "fe_vars": list(fe_vars)})
    return results

In [48]:
def build_quantile_grid_configs(
        candidate_binning_vars: list[str],
        candidate_bin_counts: list[int],
        candidate_x_vars: list[str],
        candidate_fe_vars: list[str],
        x_var_length: int = 2,
        binner_extra_grid: dict | list[dict] | None = None,
) -> list[dict[str, Any]]:
    """
    Produce configs for MultiQuantileBinner sweeping:
      - which vars to bin on
      - how many bins
      - x/fe combinations (disjoint from binned vars)
      - optional extra binner kwargs via dict-of-lists or list-of-dicts

    Parameters
    ----------
    candidate_binning_vars : list[str]
        Variables to be binned.
    candidate_bin_counts : list[int]
        Number of bins to create for each variable.
    candidate_x_vars : list[str]
        Variables to use as predictors (x_vars).
    candidate_fe_vars : list[str]
        Variables to use as fixed effects (fe_vars).
    x_var_length : int
        Number of x_vars to include in each combination.
    binner_extra_grid : dict | list[dict] | None
        Optional extra parameters for the binner.

    Returns
    -------
    list[dict[str, Any]]
        A list of configuration dictionaries for the binner.
    """
    if not candidate_binning_vars:
        return []
    if not candidate_bin_counts:
        return []

    def _expand(grid):
        if grid is None:
            return [dict()]
        if isinstance(grid, list):
            return [dict(d) for d in grid]
        if isinstance(grid, dict):
            keys = list(grid.keys())
            vals = [list(v) if isinstance(v, (list, tuple, set)) else [v] for v in (grid[k] for k in keys)]
            return [dict(zip(keys, combo)) for combo in product(*vals)]
        raise TypeError("binner_extra_grid must be a dict or list of dicts")

    extra_list = _expand(binner_extra_grid)
    configs: list[dict[str, Any]] = []

    # compute once (perf)
    x_fe_grid = build_x_fe_combinations_disjoint(
        candidate_x_vars, candidate_fe_vars, x_var_length=x_var_length
    )

    for bin_vars in all_nonempty_subsets(candidate_binning_vars):
        bset = set(bin_vars)
        for bin_count in candidate_bin_counts:
            if int(bin_count) < 2:
                continue
            bin_spec = {v: int(bin_count) for v in bin_vars}

            for combo in x_fe_grid:
                if not set(combo["x_vars"]).isdisjoint(bset):
                    continue
                for extra in extra_list:
                    binner_kwargs = {"bin_specs": bin_spec, **extra}

                    # label suffix for clarity in logs
                    tag_bits = []
                    pol = extra.get("oob_policy")
                    if pol: tag_bits.append(f"oob{pol}")
                    rate = extra.get("max_oob_rate")
                    if rate is not None: tag_bits.append(f"rate{float(rate):g}")
                    tag = f"__{'_'.join(tag_bits)}" if tag_bits else ""

                    configs.append({
                        "binner_class": MultiQuantileBinner,
                        "binner_kwargs": binner_kwargs,
                        "label": (
                            f"qbin_{bin_count}_{'-'.join(bin_vars)}"
                            f"__x_{'-'.join(combo['x_vars'])}"
                            f"__fe_{'-'.join(combo['fe_vars'])}{tag}"
                        ),
                        "x_vars": combo["x_vars"],
                        "fe_vars": combo["fe_vars"],
                    })
    return configs

In [49]:
def build_median_binner_configs(
    candidate_binning_vars: list[str],
    candidate_x_vars: list[str],
    candidate_fe_vars: list[str],
    x_var_length: int = 2,
    max_fe_len: int | None = None,
    binner_extra_grid: dict | list[dict] | None = None,
) -> list[dict[str, Any]]:
    """
    Produce configs for MultiMedianBinner sweeping subsets of variables and x/fe combos.

    Parameters
    ----------
    candidate_binning_vars : list[str]
        Variables to be binned.
    candidate_x_vars : list[str]
        Variables to use as predictors (x_vars).
    candidate_fe_vars : list[str]
        Variables to use as fixed effects (fe_vars).
    x_var_length : int
        Number of x_vars to include in each combination.
    max_fe_len : int | None
        Maximum number of fixed effects to include in each combination.
    binner_extra_grid : dict | list[dict] | None
        Optional extra parameters for the binner.

    Returns
    -------
    list[dict[str, Any]]
        A list of configuration dictionaries for the binner.
    """
    if not candidate_binning_vars:
        return []

    def _expand(grid):
        if grid is None:
            return [dict()]
        if isinstance(grid, list):
            return [dict(d) for d in grid]
        if isinstance(grid, dict):
            keys = list(grid.keys())
            vals = [ (v if isinstance(v, (list, tuple, set)) else [v]) for v in grid.values() ]
            return [dict(zip(keys, combo)) for combo in product(*vals)]
        raise TypeError("binner_extra_grid must be a dict or list of dicts")

    extra_list = _expand(binner_extra_grid)

    configs: list[dict[str, Any]] = []
    x_fe_grid = build_x_fe_combinations_disjoint(
        candidate_x_vars, candidate_fe_vars, x_var_length=x_var_length, max_fe_len=max_fe_len
    )

    for bin_vars in all_nonempty_subsets(candidate_binning_vars):
        bset = set(bin_vars)
        for combo in x_fe_grid:
            if not set(combo["x_vars"]).isdisjoint(bset):
                continue
            for extra in extra_list:
                binner_kwargs = {
                    "variables": bin_vars,
                    "group_col_name": "median_group_id",
                    "retain_flags": True,
                    **extra,
                }
                tag_bits = []
                if "retain_flags" in extra:
                    tag_bits.append(f"rf{int(bool(extra['retain_flags']))}")
                for k, v in extra.items():
                    if k == "retain_flags":
                        continue
                    tag_bits.append(f"{k}{v}")
                tag = f"__{'_'.join(tag_bits)}" if tag_bits else ""

                configs.append({
                    "binner_class": MultiMedianBinner,
                    "binner_kwargs": binner_kwargs,
                    "label": (
                        f"median_{'-'.join(bin_vars)}"
                        f"__x_{'-'.join(combo['x_vars'])}"
                        f"__fe_{'-'.join(combo['fe_vars'])}{tag}"
                    ),
                    "x_vars": combo["x_vars"],
                    "fe_vars": combo["fe_vars"],
                })
    return configs

### New Development

#### Feature Transformers

In [50]:
class WindDirToCyclic(BaseEstimator, TransformerMixin):
    def __init__(self, dir_col: str = "wind_direction_meteorological",
                 out_sin: str = "wind_dir_sin", out_cos: str = "wind_dir_cos",
                 drop_original: bool = True):
        self.dir_col = dir_col
        self.out_sin = out_sin
        self.out_cos = out_cos
        self.drop_original = drop_original

    def fit(self, X, y=None):
        if self.dir_col not in X.columns:
            raise KeyError(f"'{self.dir_col}' not in input")
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        df = X.copy()
        theta = np.deg2rad(df[self.dir_col].astype(float))
        df[self.out_sin] = np.sin(theta)
        df[self.out_cos] = np.cos(theta)
        if self.drop_original:
            df = df.drop(columns=[self.dir_col])
        return df

    def get_feature_names_out(self, input_features=None):
        base = [] if input_features is None else list(input_features)
        if self.drop_original and self.dir_col in base:
            base.remove(self.dir_col)
        return np.array(base + [self.out_sin, self.out_cos])

In [51]:
class WindDirCyclical(BaseEstimator, TransformerMixin):
    def __init__(self, col="wind_direction_meteorological"):
        self.col = col
    def fit(self, X, y=None):
        if self.col not in X.columns:
            raise KeyError(f"{self.col} not in columns")
        return self
    def transform(self, X):
        df = X.copy()
        rad = np.deg2rad(df[self.col].astype(float))
        df["wind_dir_sin"] = np.sin(rad)
        df["wind_dir_cos"] = np.cos(rad)
        return df


In [52]:
class Log1pTransform(BaseEstimator, TransformerMixin):
    def __init__(self, columns: Sequence[str]):
        self.columns = list(columns)

    def fit(self, X, y=None):
        for c in self.columns:
            if c not in X.columns:
                raise KeyError(f"'{c}' not found")
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        df = X.copy()
        for c in self.columns:
            df[c] = np.log1p(df[c].astype(float))
        return df

In [53]:
class Winsorize(BaseEstimator, TransformerMixin):
    def __init__(self, columns: Sequence[str], lower: float = 0.0, upper: float = 0.995):
        self.columns = list(columns)
        self.lower = float(lower)
        self.upper = float(upper)
        self.bounds_: dict[str, tuple[float, float]] = {}

    def fit(self, X, y=None):
        self.bounds_.clear()
        for c in self.columns:
            if c not in X.columns:
                raise KeyError(f"'{c}' not found")
            arr = X[c].astype(float).to_numpy()
            lo = np.nanpercentile(arr, self.lower * 100.0)
            hi = np.nanpercentile(arr, self.upper * 100.0)
            self.bounds_[c] = (lo, hi)
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        df = X.copy()
        for c, (lo, hi) in self.bounds_.items():
            df[c] = df[c].clip(lo, hi)
        return df

In [54]:
class StandardizeContinuous(BaseEstimator, TransformerMixin):
    def __init__(self, columns: Sequence[str], suffix: str = "_std", with_mean: bool = True, with_std: bool = True):
        self.columns = list(columns)
        self.suffix = suffix
        self.with_mean = with_mean
        self.with_std = with_std
        self.means_: dict[str, float] = {}
        self.stds_: dict[str, float] = {}

    def fit(self, X, y=None):
        self.means_.clear()
        self.stds_.clear()
        for c in self.columns:
            if c not in X.columns:
                raise KeyError(f"'{c}' not found")
            vals = X[c].astype(float)
            mu = float(vals.mean()) if self.with_mean else 0.0
            sd = float(vals.std(ddof=0)) if self.with_std else 1.0
            if sd == 0.0:
                sd = 1.0
            self.means_[c] = mu
            self.stds_[c] = sd
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        df = X.copy()
        for c in self.columns:
            mu, sd = self.means_[c], self.stds_[c]
            df[f"{c}{self.suffix}"] = (df[c].astype(float) - mu) / sd
        return df

    # helpers for invertibility
    def original_from_std(self, std_values: pd.Series, col: str) -> pd.Series:
        mu, sd = self.means_[col], self.stds_[col]
        return std_values * sd + mu

    def get_feature_names_out(self, input_features=None):
        base = [] if input_features is None else list(input_features)
        return np.array(base + [f"{c}{self.suffix}" for c in self.columns])

In [55]:
class OneHotTimeFE(BaseEstimator, TransformerMixin):
    def __init__(self, columns: Sequence[str] = ("month","hour"), drop_first: bool = True, dtype="int8"):
        self.columns = list(columns)
        self.drop_first = drop_first
        self.dtype = dtype
        self.out_cols_: list[str] = []

    def fit(self, X, y=None):
        for c in self.columns:
            if c not in X.columns:
                raise KeyError(f"'{c}' not found")
        dummies = pd.get_dummies(X[self.columns].astype("Int64"), columns=self.columns,
                                 drop_first=self.drop_first, dtype=self.dtype)
        self.out_cols_ = list(dummies.columns)
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        df = X.copy()
        dummies = pd.get_dummies(df[self.columns].astype("Int64"), columns=self.columns,
                                 drop_first=self.drop_first, dtype=self.dtype)
        # align to training columns
        for c in self.out_cols_:
            if c not in dummies.columns:
                dummies[c] = 0
        dummies = dummies[self.out_cols_]
        df = pd.concat([df, dummies], axis=1)
        return df

    def get_feature_names_out(self, input_features=None):
        base = [] if input_features is None else list(input_features)
        return np.array(base + self.out_cols_)

#### Models

In [56]:
class _BaseGAMEstimator(BaseEstimator, TransformerMixin):
    """
    Base: handles fit/transform/predict, builds X-matrix, and rescales ME using chain rule.
    """
    def __init__(self,
                 y_var: str = "tons_co2",
                 q_col: str = "demand_met",
                 q_std_col: str = "demand_met_std",         # provided by StandardizeContinuous
                 weather_linear_cols: Optional[Sequence[str]] = None,
                 fe_cols: Optional[Sequence[str]] = None,   # one-hot time FE columns
                 n_splines_q: int = 15,
                 lam_q: float = 10.0,
                 random_state: Optional[int] = 12):
        self.y_var = y_var
        self.q_col = q_col
        self.q_std_col = q_std_col
        self.weather_linear_cols = list(weather_linear_cols or [])
        self.fe_cols = list(fe_cols or [])
        self.n_splines_q = int(n_splines_q)
        self.lam_q = float(lam_q)
        self.random_state = random_state

        # learned
        self.gam_: Optional[LinearGAM] = None
        self._x_cols_: list[str] = []
        self._q_index_: Optional[int] = None
        self._q_std_: Optional[float] = None   # for chain rule
        self._q_mean_: Optional[float] = None  # not needed but stored for completeness

    # to be implemented by subclasses
    def _gam_terms(self) -> list:
        raise NotImplementedError

    def _build_matrix(self, df: pd.DataFrame) -> np.ndarray:
        cols = [self.q_std_col] + self.weather_linear_cols + self.fe_cols
        self._x_cols_ = cols
        X = df[cols].to_numpy(dtype=float)
        # remember where q_std is located for derivative
        self._q_index_ = 0
        return X

    def fit(self, X: pd.DataFrame, y: pd.Series | pd.DataFrame | None = None):
        if y is None:
            if self.y_var not in X.columns:
                raise KeyError(f"y_var '{self.y_var}' not in X")
            y_arr = X[self.y_var].astype(float).to_numpy()
        else:
            y_arr = (y.iloc[:,0] if isinstance(y, pd.DataFrame) else y).astype(float).to_numpy()

        # chain rule scale: we need std of the ORIGINAL q to convert d/dQ_std -> d/dQ
        # Expect that a StandardizeContinuous step created '<q_col>_std' and stored stats.
        # We fetch std and mean from columns if present as attributes on a fitted StandardizeContinuous,
        # but since we're inside the estimator, require caller to provide q_std_col and pass the raw q_col too.
        if self.q_col not in X.columns or self.q_std_col not in X.columns:
            raise KeyError(f"Expect both '{self.q_col}' and '{self.q_std_col}' in the DataFrame")

        # compute std directly from train (same as preprocessor stored); we keep it here for ME scaling
        q_series = X[self.q_col].astype(float)
        self._q_mean_ = float(q_series.mean())
        q_std = float(q_series.std(ddof=0))
        self._q_std_ = 1.0 if q_std == 0.0 else q_std

        Xmat = self._build_matrix(X)

        # build GAM with specified terms
        terms = self._gam_terms()
        np.random.seed(self.random_state)  # ensures reproducible smoothers
        gam = LinearGAM(terms, fit_intercept=True, lam=self.lam_q)
        gam.n_splines[self._q_index_] = int(self.n_splines_q)  # set for Q spline
        self.gam_ = gam.fit(Xmat, y_arr)
        return self

    def _derivative_q_std_df(self, df: pd.DataFrame, h: float = 1e-4) -> np.ndarray:
        """
        Version-agnostic derivative: central finite difference on the standardized demand feature.
        Returns dY/d(Q_std) for each row.
        """
        assert self.gam_ is not None, "Model not fitted"

        df_p = df.copy()
        df_m = df.copy()
        df_p[self.q_std_col] = df_p[self.q_std_col].astype(float) + h
        df_m[self.q_std_col] = df_m[self.q_std_col].astype(float) - h

        Xp = self._build_matrix(df_p)
        Xm = self._build_matrix(df_m)

        # predict is vectorized; two passes only
        y_p = self.gam_.predict(Xp)
        y_m = self.gam_.predict(Xm)
        return (y_p - y_m) / (2.0 * h)

    def predict(self, X: pd.DataFrame, predict_type: str = "ME") -> np.ndarray:
        if self.gam_ is None:
            raise RuntimeError("Model not fitted")

        if predict_type.lower() == "y":
            Xmat = self._build_matrix(X)
            return self.gam_.predict(Xmat)

        # dY/d(Q_std) via central differences, then chain rule to original Q units
        d_y_d_qstd = self._derivative_q_std_df(X)              # length n
        q_sd = float(self._q_std_ or 1.0)                      # from fit()
        return d_y_d_qstd * (1.0 / q_sd)                       # dY/dQ = dY/dQstd * (1/std(Q))

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        """
        Returns X plus columns:
            - 'y_pred' : predicted tons_co2
            - 'ME'     : marginal emissions in original Q units
        """
        df = X.copy()
        yhat = self.predict(df, predict_type="y")
        me = self.predict(df, predict_type="ME")
        df["y_pred"] = yhat
        df["ME"] = me
        return df

    def get_params(self, deep=True):
        return {
            "y_var": self.y_var,
            "q_col": self.q_col,
            "q_std_col": self.q_std_col,
            "weather_linear_cols": list(self.weather_linear_cols),
            "fe_cols": list(self.fe_cols),
            "n_splines_q": self.n_splines_q,
            "lam_q": self.lam_q,
            "random_state": self.random_state,
        }

    def set_params(self, **params):
        for k, v in params.items():
            setattr(self, k, v)
        return self

In [57]:
class QSplineRegressorPyGAM(_BaseGAMEstimator):
    def _gam_terms(self):
            lin_count = len(self.weather_linear_cols) + len(self.fe_cols)
            terms_list = [s(0)] + [l(i+1) for i in range(lin_count)]
            return _sum_terms(terms_list)  # <-- return a TermList, not a list


In [58]:
class QGAMRegressorPyGAM(_BaseGAMEstimator):
    def __init__(self,
                 y_var: str = "tons_co2",
                 q_col: str = "demand_met",
                 q_std_col: str = "demand_met_std",
                 # which of the linear cols should be smoothed (pass their *_std names)
                 smooth_cols: Optional[Sequence[str]] = None,   # e.g., ["surface_net_solar_radiation_joules_per_m2_std","wind_speed_mps_std","temperature_celsius_std"]
                 weather_linear_cols: Optional[Sequence[str]] = None,
                 fe_cols: Optional[Sequence[str]] = None,
                 n_splines_q: int = 15, lam_q: float = 10.0,
                 n_splines_other: int = 15, lam_other: float = 10.0,
                 random_state: Optional[int] = 12):
        super().__init__(y_var=y_var, q_col=q_col, q_std_col=q_std_col,
                         weather_linear_cols=weather_linear_cols, fe_cols=fe_cols,
                         n_splines_q=n_splines_q, lam_q=lam_q, random_state=random_state)
        self.smooth_cols = list(smooth_cols or [])
        self.n_splines_other = int(n_splines_other)
        self.lam_other = float(lam_other)

    def _gam_terms(self):
        terms_list = [s(0)]
        offset = 1
        for i, col in enumerate(self.weather_linear_cols + self.fe_cols):
            j = offset + i
            if col in self.smooth_cols:
                terms_list.append(s(j))
            else:
                terms_list.append(l(j))
        return _sum_terms(terms_list)

    def fit(self, X: pd.DataFrame, y: pd.Series | pd.DataFrame | None = None):
        # need to override to set different n_splines / lam for "other" smooth terms
        super().fit(X, y)  # builds self.gam_, self._x_cols_, etc.
        # Refit using desired lam per-term
        if self.gam_ is None:
            return self

        Xmat = self._build_matrix(X)
        # Recreate terms and assign lam/n_splines per term
        terms = self._gam_terms()
        np.random.seed(self.random_state)  # ensures reproducible smoothers
        gam = LinearGAM(terms, fit_intercept=True)
        # lam vector length = number of terms
        lam_vec = []
        for idx, term in enumerate(terms):
            if idx == 0:  # Q spline
                lam_vec.append(self.lam_q)
            else:
                # check which column
                colname = (self.weather_linear_cols + self.fe_cols)[idx - 1]
                lam_vec.append(self.lam_other if colname in self.smooth_cols else 0.0)
        gam.lam = lam_vec

        # set n_splines for smooth terms
        # term indices align with features order; check types via isinstance on term
        from pygam.terms import SplineTerm, LinearTerm
        for idx, term in enumerate(gam.terms):
            if isinstance(term, SplineTerm):
                if idx == 0:  # Q
                    gam.n_splines[idx] = int(self.n_splines_q)
                else:
                    gam.n_splines[idx] = int(self.n_splines_other)

        y_arr = (X[self.y_var].astype(float).to_numpy()
                 if y is None else (y.iloc[:,0] if isinstance(y, pd.DataFrame) else y).astype(float).to_numpy())
        np.random.seed(self.random_state)          # for reproducibility
        self.gam_ = gam.fit(Xmat, y_arr)
        return self

#### Pipelines

In [59]:
def build_mef_preprocess_pipeline(
    timestamp_col: str = "timestamp",
    use_winsorize_precip: bool = True,
    winsor_upper: float = 0.995,
):
    """
    Returns a sklearn Pipeline that:
      - adds month/hour (using your DateTimeFeatureAdder)
      - cyclic-encodes wind direction
      - log1p precipitation (winsorize optional)
      - standardizes continuous columns (adds *_std, keeps originals)
      - one-hot month/hour
    """
    # you already have DateTimeFeatureAdder in your codebase; reuse it here
    dt = DateTimeFeatureAdder(timestamp_col=timestamp_col, drop_original=False)

    # columns from your reduced set
    cont_cols = [
        "demand_met",
        "wind_speed_mps",
        "temperature_celsius",
        "precipitation_mm",   # will be log1p'ed first
        "surface_net_solar_radiation_joules_per_m2",
        "total_cloud_cover",
    ]

    steps = [
        ("DateTime", dt),
        ("WindDirCyclic", WindDirToCyclic(dir_col="wind_direction_meteorological",
                                          out_sin="wind_dir_sin", out_cos="wind_dir_cos", drop_original=True)),
        ("Log1pPrecip", Log1pTransform(columns=["precipitation_mm"])),
    ]
    if use_winsorize_precip:
        steps.append(("WinsorizePrecip", Winsorize(columns=["precipitation_mm"], lower=0.0, upper=winsor_upper)))

    steps += [
        ("Standardize", StandardizeContinuous(columns=cont_cols, suffix="_std")),
        ("TimeFE", OneHotTimeFE(columns=("month","hour"), drop_first=True, dtype="int8")),
    ]
    return Pipeline(steps, verbose=False)

In [60]:
def build_mef_preprocess_pipeline_2(
    timestamp_col: str = "timestamp",
    use_winsorize_precip: bool = True,
    winsor_upper: float = 0.995,
):
    cont_cols = [
        "demand_met",
        "wind_speed_mps",
        "temperature_celsius",
        "precipitation_mm_log1p",  # <- we’ll create this
        "surface_net_solar_radiation_joules_per_m2",
        "total_cloud_cover",
    ]

    steps = [
        ("Add_Datetime_Features", DateTimeFeatureAdder(timestamp_col=timestamp_col, drop_original=False)),
        ("Add_Fourier_Time", TimeFourierAdder(timestamp_col=timestamp_col)),        # hour/dow/doy sin/cos
        ("WindDirCyc", WindDirCyclical(col="wind_direction_meteorological")),      # sin/cos of wind dir
        ("Add_Original", AnalysisFeatureAdder(timestamp_col=timestamp_col, demand_met_col="demand_met", co2_col="tons_co2")),
    ]

    if use_winsorize_precip:
        steps.append(("WinsorizePrecip", Winsorize(columns=["precipitation_mm"], lower=0.0, upper=winsor_upper)))
    steps.append(("Log1pPrecip", Log1pColumns(columns=["precipitation_mm"])))

    steps += [
        ("Standardize", StandardizeContinuous(columns=cont_cols, suffix="_std")),  # adds *_std, keeps originals
        # No month/hour one-hots here — Fourier + is_weekend are enough
    ]

    return Pipeline(steps, verbose=False)


In [61]:
def build_qspline_pipeline(
    preprocess: Pipeline,
    fe_cols_prefixes: Iterable[str] = ("month_", "hour_"),
    y_var: str = "tons_co2",
    q_col: str = "demand_met",
    q_std_col: str = "demand_met_std",
    weather_linear_base: Optional[Sequence[str]] = None,
    n_splines_q: int = 15,
    lam_q: float = 10.0,
    random_state: int = 12,
):
    """
    Full pipeline: [preprocess] -> QSplineRegressorPyGAM
    """
    weather_linear_base = list(weather_linear_base or [])
    # auto-discover FE one-hots produced by OneHotTimeFE
    def _fe_names_after_fit(df_pre: pd.DataFrame) -> list[str]:
        return [c for c in df_pre.columns if c.startswith(fe_cols_prefixes)]

    # placeholder estimator, fe_cols filled at fit-time
    est = QSplineRegressorPyGAM(
        y_var=y_var, q_col=q_col, q_std_col=q_std_col,
        weather_linear_cols=weather_linear_base + ["wind_dir_sin", "wind_dir_cos"],
        fe_cols=[], n_splines_q=n_splines_q, lam_q=lam_q, random_state=random_state,
    )

    class _AttachFENames(BaseEstimator, TransformerMixin):
        """Small meta-transformer to attach FE names right before fitting the estimator inside Pipeline."""
        def __init__(self, estimator: QSplineRegressorPyGAM):
            self.estimator = estimator
        def fit(self, X, y=None):
            # X here is already preprocessed; capture FE names
            fe_cols = _fe_names_after_fit(X)
            self.estimator.fe_cols = fe_cols
            return self
        def transform(self, X):
            return X

    return Pipeline([
        ("PreprocessMEF", preprocess),
        ("AttachFE", _AttachFENames(est)),
        ("QSplineRegressorPyGAM", est),
    ])


In [62]:
def build_qgam_pipeline(
    preprocess: Pipeline,
    fe_cols_prefixes: Iterable[str] = ("month_", "hour_"),
    y_var: str = "tons_co2",
    q_col: str = "demand_met",
    q_std_col: str = "demand_met_std",
    smooth_cols: Optional[Sequence[str]] = None,  # e.g., ["surface_net_solar_radiation_joules_per_m2_std","wind_speed_mps_std","temperature_celsius_std"]
    linear_extra: Optional[Sequence[str]] = None, # e.g., ["precipitation_mm_std","total_cloud_cover_std","wind_dir_sin","wind_dir_cos"]
    n_splines_q: int = 15, lam_q: float = 10.0,
    n_splines_other: int = 15, lam_other: float = 10.0,
    random_state: int = 12,
):
    smooth_cols = list(smooth_cols or [])
    linear_extra = list(linear_extra or ["precipitation_mm_std", "total_cloud_cover_std", "wind_dir_sin", "wind_dir_cos"])

    def _fe_names_after_fit(df_pre: pd.DataFrame) -> list[str]:
        return [c for c in df_pre.columns if c.startswith(fe_cols_prefixes)]

    est = QGAMRegressorPyGAM(
        y_var=y_var, q_col=q_col, q_std_col=q_std_col,
        smooth_cols=smooth_cols,
        weather_linear_cols=smooth_cols + linear_extra,   # order matters; smooth subset will be detected inside
        fe_cols=[],
        n_splines_q=n_splines_q, lam_q=lam_q,
        n_splines_other=n_splines_other, lam_other=lam_other,
        random_state=random_state,
    )

    class _AttachFENames(BaseEstimator, TransformerMixin):
        def __init__(self, estimator: QGAMRegressorPyGAM):
            self.estimator = estimator
        def fit(self, X, y=None):
            self.estimator.fe_cols = _fe_names_after_fit(X)
            return self
        def transform(self, X):
            return X

    return Pipeline([
        ("PreprocessMEF", preprocess),
        ("AttachFE", _AttachFENames(est)),
        ("QGAMRegressorPyGAM", est),
    ])

In [63]:
class TimeFourierAdder(BaseEstimator, TransformerMixin):
    """
    Add smooth cyclic time features:
      - hour_sin, hour_cos   (24h cycle using hour + minute)
      - dow_sin,  dow_cos    (weekly cycle, 0=Mon..6=Sun)
      - doy_sin,  doy_cos    (annual cycle, 1..365/366)
    Assumes a column `timestamp` (or override via timestamp_col).
    """
    def __init__(self, timestamp_col="timestamp",
                 add_hour=True, add_week=True, add_doy=True):
        self.timestamp_col = timestamp_col
        self.add_hour = add_hour
        self.add_week = add_week
        self.add_doy = add_doy

    def fit(self, X, y=None):
        if self.timestamp_col not in X.columns:
            raise KeyError(f"{self.timestamp_col} not in columns")
        return self

    def transform(self, X):
        df = X.copy()
        t = pd.to_datetime(df[self.timestamp_col], errors="raise")

        if self.add_hour:
            h = t.dt.hour + t.dt.minute/60.0
            df["hour_sin"] = np.sin(2*np.pi*h/24.0)
            df["hour_cos"] = np.cos(2*np.pi*h/24.0)

        if self.add_week:
            dow = t.dt.dayofweek.astype(float)  # 0=Mon
            df["dow_sin"] = np.sin(2*np.pi*dow/7.0)
            df["dow_cos"] = np.cos(2*np.pi*dow/7.0)

        if self.add_doy:
            doy = t.dt.dayofyear.astype(float)
            df["doy_sin"] = np.sin(2*np.pi*doy/365.0)   # 365 is fine for our purpose
            df["doy_cos"] = np.cos(2*np.pi*doy/365.0)

        return df


In [64]:
def evaluate_gam_pipeline(fitted_pipeline: Pipeline, X: pd.DataFrame, y: pd.Series,
                          id_cols: Sequence[str] = ("timestamp","city")) -> pd.DataFrame:
    """
    Apply a fitted [preprocess -> (Q)GAM] and return a tidy frame with:
        id_cols ∩ cols, y_true, y_pred, ME
    """
    # push through whole pipeline (model's transform attaches y_pred, ME)
    Z = fitted_pipeline.transform(pd.concat([X, y.rename("tons_co2")], axis=1))
    cols = [c for c in id_cols if c in Z.columns] + ["y_pred","ME"]
    out = pd.DataFrame(index=Z.index)
    for c in cols:
        out[c] = Z[c]
    out["y_true"] = y.values
    return out

In [65]:
def metrics_from_frame(df: pd.DataFrame) -> dict:
    y_true = df["y_true"].to_numpy(float)
    y_pred = df["y_pred"].to_numpy(float)
    return {
        "r2": float(r2_score(y_true, y_pred)),
        "rmse": float(root_mean_squared_error(y_true, y_pred)),
        "mae": float(mean_absolute_error(y_true, y_pred)),
        "n_obs": int(len(df)),
    }

In [66]:
# helper
def _sum_terms(terms):
    # terms: list like [s(0), l(1), ...]
    return reduce(add, terms)


In [67]:
class Log1pColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = list(columns)
    def fit(self, X, y=None):
        missing = [c for c in self.columns if c not in X.columns]
        if missing: raise KeyError(f"Missing for log1p: {missing}")
        return self
    def transform(self, X):
        df = X.copy()
        for c in self.columns:
            df[f"{c}_log1p"] = np.log1p(df[c].astype(float).clip(lower=0))
        return df

## Loading Data

#### Directories

In [68]:
# DIRECTORIES AND PATHS
# This is a redundant code block, but it is included as a reminder of the directory variables.
base_data_directory = "data"  # Base directory where the dataframes will be saved
hitachi_data_directory = os.path.join(base_data_directory, "hitachi_copy")  # Directory where the dataframes will be saved
marginal_emissions_development_directory = os.path.join(base_data_directory, "marginal_emissions_development")  # Directory for marginal emissions development data
marginal_emissions_results_directory = os.path.join(marginal_emissions_development_directory, "results")
marginal_emissions_logs_directory = os.path.join(marginal_emissions_development_directory, "logs")

marginal_emissions_prefix = "marginal_emissions_results"

In [69]:
print("\n" + "-" * 120)
print(f"Contents of '{hitachi_data_directory}' and subdirectories:\n" + "-" * 120)
for root, dirs, files in os.walk(hitachi_data_directory):
    for f in sorted(files):
        rel_dir = os.path.relpath(root, hitachi_data_directory)
        rel_file = os.path.join(rel_dir, f) if rel_dir != "." else f
        print(f"  - {rel_file}")


------------------------------------------------------------------------------------------------------------------------
Contents of 'data/hitachi_copy' and subdirectories:
------------------------------------------------------------------------------------------------------------------------
  - .DS_Store
  - customers_20250714_1401.parquet
  - grid_readings_20250714_1401.parquet
  - grid_readings_20250714_1401_processed.parquet
  - grid_readings_20250714_1401_processed_half_hourly.parquet
  - weather_20250714_1401.parquet
  - weather_20250714_1401_processed.parquet
  - weather_and_grid_data_half-hourly_20250714_1401.parquet
  - weather_data_combined_20250714_1401.parquet
  - meter_primary_files/.DS_Store
  - meter_primary_files/meter_readings_2021_20250714_2015.parquet
  - meter_primary_files/meter_readings_2021_20250714_2015_formatted.parquet
  - meter_primary_files/meter_readings_2021_Q4_20250714_2015_formatted.parquet
  - meter_primary_files/meter_readings_2022_20250714_2324.parq

#### File Paths

In [70]:
# cleaned weather data
base_file = "weather_and_grid_data_half-hourly_20250714_1401"

train_file = "marginal_emissions_estimation_20250714_1401_train_data"
validation_file = "marginal_emissions_estimation_20250714_1401_validation_data"
test_file = "marginal_emissions_estimation_20250714_1401_test_data"

In [71]:
base_filepath = os.path.join(hitachi_data_directory, base_file + ".parquet")

train_filepath = os.path.join(marginal_emissions_development_directory, train_file + ".parquet")
validation_filepath = os.path.join(marginal_emissions_development_directory, validation_file + ".parquet")
test_filepath = os.path.join(marginal_emissions_development_directory, test_file + ".parquet")

#### Load and Look at Data

In [72]:
base_pldf = pl.read_parquet(base_filepath)

In [73]:
train_pldf = pl.read_parquet(train_filepath)
validation_pldf = pl.read_parquet(validation_filepath)
test_pldf = pl.read_parquet(test_filepath)

In [74]:
# Sample Rows of the DataFrame
print("\n" + "-" * 120)
print(f"Sample rows of prepared dataset [train_pldf]:\n" + "-" * 120)
display(train_pldf.sample(8))
display(train_pldf.schema)


------------------------------------------------------------------------------------------------------------------------
Sample rows of prepared dataset [train_pldf]:
------------------------------------------------------------------------------------------------------------------------


timestamp,city,land_latitude,land_longitude,wind_speed_mps,wind_direction_meteorological,temperature_celsius,precipitation_mm,surface_net_solar_radiation_kWh_per_m2,surface_solar_radiation_downwards_kWh_per_m2,surface_net_solar_radiation_joules_per_m2,surface_solar_radiation_downwards_joules_per_m2,total_cloud_cover,high_cloud_cover,medium_cloud_cover,low_cloud_cover,thermal_generation,gas_generation,hydro_generation,nuclear_generation,renewable_generation,total_generation,demand_met,non_renewable_generation,tons_co2,g_co2_per_kwh,tons_co2_per_mwh,wind_dir_cardinal_8,wind_dir_cardinal_16,wind_dir_cardinal_4
"datetime[μs, Asia/Kolkata]",cat,f64,f64,f32,f32,f64,f64,f64,f64,f64,f64,f32,f32,f32,f32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str
2021-10-08 15:00:00 IST,"""delhi""",28.6,76.94,2.140953,25.835144,26.379761,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,114385.583333,7599.0,20366.416667,4383.75,23636.666667,170371.416667,169058.166667,146734.75,57438.4312,674.281609,0.674282,"""NE""","""NNE""","""N"""
2022-04-02 07:30:00 IST,"""delhi""",28.5,76.94,1.252711,269.7388,37.496246,0.0,379635.698365,478384.166353,1.3667e6,1.7222e6,0.0,0.0,0.0,0.0,148525.25,3716.583333,17398.083333,4775.583333,7029.416667,181444.916667,179586.333333,174415.5,73197.73305,806.829505,0.80683,"""W""","""W""","""W"""
2023-06-03 02:00:00 IST,"""delhi""",28.5,77.24,1.250598,4.888458,26.310791,0.0,31047.207561,37781.735594,111762.140077,136002.537948,0.0,0.0,0.0,0.0,152663.333333,2826.083333,18549.166667,4306.75,14895.0,193240.333333,191291.5,178345.333333,75013.44475,776.375407,0.776375,"""N""","""N""","""N"""
2021-12-22 03:30:00 IST,"""delhi""",28.5,77.04,0.308357,305.025024,12.031494,0.0,31733.148774,38559.853892,114239.335587,138815.474012,0.477142,0.477142,0.0,0.0,107928.25,2750.333333,7460.0,5760.916667,496.5,124396.0,122301.916667,123899.5,53201.4684,855.356809,0.855357,"""NW""","""NW""","""W"""
2022-09-01 18:00:00 IST,"""delhi""",28.8,77.24,1.668718,167.790649,28.773071,0.0,0.0,0.0,0.0,0.0,0.998016,0.998016,0.085846,0.035889,131903.5,3079.083333,33746.333333,4913.25,8871.166667,182513.333333,180045.666667,173642.166667,64956.17895,711.801991,0.711802,"""S""","""SSE""","""S"""
2021-02-04 21:00:00 IST,"""delhi""",28.6,77.04,1.573774,337.806122,9.36554,0.0,0.0,0.0,0.0,0.0,0.895966,0.0,0.0,0.895966,128295.166667,4127.333333,9463.166667,3943.333333,6764.333333,152593.333333,153224.166667,145829.0,63434.0552,831.422076,0.831422,"""N""","""NNW""","""N"""
2022-07-02 19:00:00 IST,"""mumbai""",18.6,72.97,5.224999,224.165146,25.368011,0.262992,0.0,0.0,0.0,0.0,0.986664,0.949738,0.348267,0.412567,124265.416667,2436.416667,25907.166667,5394.5,17885.5,175889.0,173530.916667,158003.5,61090.30475,694.634387,0.694634,"""SW""","""SW""","""S"""
2023-10-31 04:00:00 IST,"""mumbai""",19.2,72.77,1.930534,101.093933,28.272888,0.0,121951.73787,135923.404761,439026.256331,489324.257141,0.016663,0.0,0.016663,0.0,139990.416667,2156.25,8066.916667,5397.833333,4847.5,160458.916667,158748.5,155611.416667,68688.48725,856.150478,0.85615,"""E""","""E""","""E"""


Schema([('timestamp', Datetime(time_unit='us', time_zone='Asia/Kolkata')),
        ('city', Categorical(ordering='physical')),
        ('land_latitude', Float64),
        ('land_longitude', Float64),
        ('wind_speed_mps', Float32),
        ('wind_direction_meteorological', Float32),
        ('temperature_celsius', Float64),
        ('precipitation_mm', Float64),
        ('surface_net_solar_radiation_kWh_per_m2', Float64),
        ('surface_solar_radiation_downwards_kWh_per_m2', Float64),
        ('surface_net_solar_radiation_joules_per_m2', Float64),
        ('surface_solar_radiation_downwards_joules_per_m2', Float64),
        ('total_cloud_cover', Float32),
        ('high_cloud_cover', Float32),
        ('medium_cloud_cover', Float32),
        ('low_cloud_cover', Float32),
        ('thermal_generation', Float64),
        ('gas_generation', Float64),
        ('hydro_generation', Float64),
        ('nuclear_generation', Float64),
        ('renewable_generation', Float64),
        (

In [75]:
# reminder of time boundaries
print("Train Time Boundaries:")
print(f"\tStart Time: {train_pldf['timestamp'].min()}")
print(f"\tEnd Time: {train_pldf['timestamp'].max()}")
print(f"\tTotal Duration: {train_pldf['timestamp'].max() - train_pldf['timestamp'].min()}")
print("\n" + "-" * 80)

Train Time Boundaries:
	Start Time: 2021-01-01 00:00:00+05:30
	End Time: 2023-12-31 23:30:00+05:30
	Total Duration: 1094 days, 23:30:00

--------------------------------------------------------------------------------


In [76]:
# reminder of time boundaries
print("Validation Time Boundaries:")
print(f"\tStart Time: {validation_pldf['timestamp'].min()}")
print(f"\tEnd Time: {validation_pldf['timestamp'].max()}")
print(f"\tTotal Duration: {validation_pldf['timestamp'].max() - validation_pldf['timestamp'].min()}")
print("\n" + "-" * 80)

Validation Time Boundaries:
	Start Time: 2024-01-01 00:00:00+05:30
	End Time: 2024-05-31 23:30:00+05:30
	Total Duration: 151 days, 23:30:00

--------------------------------------------------------------------------------


In [77]:
# reminder of time boundaries
print("Test Time Boundaries:")
print(f"\tStart Time: {test_pldf['timestamp'].min()}")
print(f"\tEnd Time: {test_pldf['timestamp'].max()}")
print(f"\tTotal Duration: {test_pldf['timestamp'].max() - test_pldf['timestamp'].min()}")
print("\n" + "-" * 80)

Test Time Boundaries:
	Start Time: 2024-06-01 00:00:00+05:30
	End Time: 2025-05-31 23:30:00+05:30
	Total Duration: 364 days, 23:30:00

--------------------------------------------------------------------------------


## Data Processing

Core demand variable (for MEF)
demand_met → spline/GAM smooth term.

Weather variables
wind_speed_mps → standardized.

wind_direction_meteorological → cyclic transform:

wind_dir_sin = sin(2π × wind_dir / 360)

wind_dir_cos = cos(2π × wind_dir / 360)

temperature_celsius → standardized.

precipitation_mm → log1p transform → standardized (possible winsorize at 99.5% after log1p).

surface_net_solar_radiation_joules_per_m2 → standardized (convert to kWh/m² if needed for interpretability).

total_cloud_cover → standardized.

Transform

precipitation_mm → log1p

wind_direction_meteorological → sin & cos cyclic encoding

Optional Winsorize

Precipitation after log1p (99.5% cutoff) to tame extreme monsoon spikes.

Standardize continuous predictors (mean 0, sd 1).

Drop any constant or NA-heavy columns (shouldn’t be an issue now).



📐 Model setup
Spline model
Formula:

CO₂_t = s(Q_t) + \beta^\top W_t + \text{FE}_{month,hour} + \epsilon_t
Where:

s(Q_t) = cubic spline on standardized demand (demand_met), 4–6 interior knots at demand quantiles.

W_t = weather predictors (linear terms after scaling).

Fixed effects for month & hour handle seasonality & diurnal cycles.

MEF: derivative of spline w.r.t Q.

GAM
Formula:

CO₂_t = s(Q_t) + s(\text{solar}) + s(\text{wind_speed}) + s(\text{temperature}) + \beta^\top W_{\text{linear}} + \text{FE}_{month,hour} + \epsilon_t
Where:

Smooth terms: demand_met, surface_net_solar_radiation_joules_per_m2, wind_speed_mps, temperature_celsius.

Linear terms: cyclic wind dir (sin, cos), log1p_precip, total_cloud_cover.

Same fixed effects as spline model.

MEF: derivative of Q-smooth from GAM.

In [160]:
# Conversion to Pandas DataFrame for compatibility with existing code
train_df = train_pldf.to_pandas()
validation_df = validation_pldf.to_pandas()
test_df = test_pldf.to_pandas()

In [161]:
print("Columns in Train DataFrame:")
print(train_df.columns.tolist())
print("\nColumns in Validation DataFrame:")
print(validation_df.columns.tolist())
print("\nColumns in Test DataFrame:")
print(test_df.columns.tolist())

Columns in Train DataFrame:
['timestamp', 'city', 'land_latitude', 'land_longitude', 'wind_speed_mps', 'wind_direction_meteorological', 'temperature_celsius', 'precipitation_mm', 'surface_net_solar_radiation_kWh_per_m2', 'surface_solar_radiation_downwards_kWh_per_m2', 'surface_net_solar_radiation_joules_per_m2', 'surface_solar_radiation_downwards_joules_per_m2', 'total_cloud_cover', 'high_cloud_cover', 'medium_cloud_cover', 'low_cloud_cover', 'thermal_generation', 'gas_generation', 'hydro_generation', 'nuclear_generation', 'renewable_generation', 'total_generation', 'demand_met', 'non_renewable_generation', 'tons_co2', 'g_co2_per_kwh', 'tons_co2_per_mwh', 'wind_dir_cardinal_8', 'wind_dir_cardinal_16', 'wind_dir_cardinal_4']

Columns in Validation DataFrame:
['timestamp', 'city', 'land_latitude', 'land_longitude', 'wind_speed_mps', 'wind_direction_meteorological', 'temperature_celsius', 'precipitation_mm', 'surface_net_solar_radiation_kWh_per_m2', 'surface_solar_radiation_downwards_kWh_

### Original Analysis

In [162]:
# marginal_emissions_results_directory
# marginal_emissions_logs_directory

# marginal_emissions_prefix

In [163]:
X_train = train_df.drop(columns=["tons_co2"])
y_train = train_df["tons_co2"]
X_val = validation_df.drop(columns=["tons_co2"])
y_val = validation_df["tons_co2"]
X_test = test_df.drop(columns=["tons_co2"])
y_test = test_df["tons_co2"]

In [164]:
# New: a plain OLS regressor that builds q^1..q^4 and q×(weather) interactions,
# then exposes predict(..., predict_type="ME") for the exact derivative wrt q.


class Poly4MERegressor(BaseEstimator, TransformerMixin):
    def __init__(self,
                 y_var="tons_co2",
                 q_col="demand_met",                # use RAW demand (not std)
                 weather_cols_std=("wind_speed_mps_std",
                                   "temperature_celsius_std",
                                   "surface_net_solar_radiation_joules_per_m2_std"),
                 wind_dir_cols=("wind_dir_sin","wind_dir_cos"),
                 extra_linear_cols=("precipitation_mm_log1p_std",
                                    "total_cloud_cover_std",
                                    "hour_sin","hour_cos",
                                    "dow_sin","dow_cos",
                                    "doy_sin","doy_cos",
                                    "is_weekend"),
                 add_interactions=True):
        self.y_var = y_var
        self.q_col = q_col
        self.weather_cols_std = list(weather_cols_std)
        self.wind_dir_cols = list(wind_dir_cols)
        self.extra_linear_cols = list(extra_linear_cols)
        self.add_interactions = bool(add_interactions)

        self.model_ = None
        self.term_names_ = None  # for convenience / introspection

    def _design(self, X: pd.DataFrame) -> pd.DataFrame:
        df = X.copy()
        q = self.q_col

        # polynomial in RAW q
        df[f"{q}_p1"] = df[q].astype(float)
        df[f"{q}_p2"] = df[q].astype(float)**2
        df[f"{q}_p3"] = df[q].astype(float)**3
        df[f"{q}_p4"] = df[q].astype(float)**4

        # interactions q × standardized weather / wind_dir
        inter_cols = []
        if self.add_interactions:
            for c in list(self.weather_cols_std) + list(self.wind_dir_cols):
                name = f"{q}_x_{c}"
                df[name] = df[q].astype(float) * df[c].astype(float)
                inter_cols.append(name)

        # stash names for predict/ME
        self._poly_cols = [f"{q}_p1", f"{q}_p2", f"{q}_p3", f"{q}_p4"]
        self._inter_cols = inter_cols
        self._lin_cols = list(self.weather_cols_std) + list(self.wind_dir_cols) + list(self.extra_linear_cols)

        return df

    def fit(self, X: pd.DataFrame, y=None):
        if y is None:
            if self.y_var not in X.columns:
                raise KeyError(f"'{self.y_var}' missing in X")
            y_ser = X[self.y_var]
        else:
            y_ser = (y if isinstance(y, pd.Series) else y.iloc[:,0])
        df = self._design(X)

        # Build formula: y ~ q_p1 + q_p2 + q_p3 + q_p4 + (q×weather) + linear controls
        rhs = self._poly_cols + self._inter_cols + self._lin_cols
        # keep only columns that actually exist (e.g. some Fourier flags might be missing)
        rhs = [c for c in rhs if c in df.columns]
        formula = f"{self.y_var} ~ " + " + ".join(rhs)

        self.model_ = smf.ols(formula, data=df.assign(**{self.y_var: y_ser})).fit()
        self.term_names_ = rhs
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        # Attach y_pred and ME (exact derivative wrt q)
        if self.model_ is None:
            raise RuntimeError("Model not fitted")
        df = self._design(X)
        yhat = self.model_.predict(df)
        me = self._me_from_df(df)
        out = df.copy()
        out["y_pred"] = np.asarray(yhat, dtype=float)
        out["ME"] = np.asarray(me, dtype=float)
        return out

    def predict(self, X: pd.DataFrame, predict_type="ME"):
        df = self._design(X)
        if predict_type.lower() == "y":
            return self.model_.predict(df)
        return self._me_from_df(df)

    # --- exact derivative wrt q ---
    def _me_from_df(self, df: pd.DataFrame) -> np.ndarray:
        q = df[self.q_col].astype(float).to_numpy()
        coefs = self.model_.params  # pandas Series (includes intercept)

        # coefficients (missing defaults to 0.0 if regularization or dropped)
        def c(name): return float(coefs[name]) if name in coefs.index else 0.0

        q1 = c(f"{self.q_col}_p1")
        q2 = c(f"{self.q_col}_p2")
        q3 = c(f"{self.q_col}_p3")
        q4 = c(f"{self.q_col}_p4")

        # base polynomial derivative: β1 + 2β2 q + 3β3 q^2 + 4β4 q^3
        me = (q1
              + 2.0 * q2 * q
              + 3.0 * q3 * (q**2)
              + 4.0 * q4 * (q**3))

        # + interactions: sum γ_c * feature_c
        for cfeat in list(self.weather_cols_std) + list(self.wind_dir_cols):
            γ = c(f"{self.q_col}_x_{cfeat}")
            if γ != 0.0 and cfeat in df.columns:
                me = me + γ * df[cfeat].astype(float).to_numpy()

        return me


In [165]:
preprocess = build_mef_preprocess_pipeline_2(timestamp_col="timestamp", use_winsorize_precip=True)

In [166]:

poly4 = Poly4MERegressor(
    y_var="tons_co2",
    q_col="demand_met",  # raw q
    weather_cols_std=("wind_speed_mps_std",
                      "temperature_celsius_std",
                      "surface_net_solar_radiation_joules_per_m2_std"),
    wind_dir_cols=("wind_dir_sin","wind_dir_cos"),
    extra_linear_cols=("precipitation_mm_log1p_std",
                       "total_cloud_cover_std",
                       "hour_sin","hour_cos",
                       "dow_sin","dow_cos",
                       "doy_sin","doy_cos",
                       "is_weekend"),
    add_interactions=True
)

poly_pipeline = Pipeline([
    ("PreprocessMEF", preprocess),
    ("Poly4ME", poly4),
])


In [167]:
# Fit on train (must pass y so the estimator can build and fit OLS)
poly_pipeline.fit(pd.concat([X_train, y_train.rename("tons_co2")], axis=1), y_train)

# Evaluate
train_eval = evaluate_gam_pipeline(poly_pipeline, X_train, y_train)
val_eval  = evaluate_gam_pipeline(poly_pipeline, X_val,  y_val)


print("TRAIN:", metrics_from_frame(train_eval))
print("VAL:",  metrics_from_frame(val_eval))

TRAIN: {'r2': 0.4823478103208152, 'rmse': 5632.8721326194045, 'mae': 4563.779819447094, 'n_obs': 2365200}
VAL: {'r2': -1.004314431223058, 'rmse': 8447.794490421793, 'mae': 7504.033985986517, 'n_obs': 328320}


In [ ]:
# --- 1) time features (Fourier) + weekend ---
def add_time_features(df, ts_col="timestamp"):
    out = df.copy()
    t = pd.to_datetime(out[ts_col], errors="raise")
    # hour-of-day as fractional hour
    h = t.dt.hour + t.dt.minute/60.0
    out["hour_sin"] = np.sin(2*np.pi*h/24.0)
    out["hour_cos"] = np.cos(2*np.pi*h/24.0)
    # day-of-week: 0=Mon..6=Sun
    dow = t.dt.dayofweek.astype(float)
    out["dow_sin"] = np.sin(2*np.pi*dow/7.0)
    out["dow_cos"] = np.cos(2*np.pi*dow/7.0)
    # day-of-year
    doy = t.dt.dayofyear.astype(float)
    out["doy_sin"] = np.sin(2*np.pi*doy/365.0)
    out["doy_cos"] = np.cos(2*np.pi*doy/365.0)
    # weekend flag
    out["is_weekend"] = (t.dt.dayofweek >= 5).astype(int)
    return out

X_train_f = add_time_features(X_train)
X_val_f   = add_time_features(X_val)

# --- 2) wind direction cyclic encoding ---
def add_wind_dir_cyclic(df, col="wind_direction_meteorological"):
    out = df.copy()
    theta = np.deg2rad(out[col].astype(float))
    out["wind_dir_sin"] = np.sin(theta)
    out["wind_dir_cos"] = np.cos(theta)
    return out

X_train_f = add_wind_dir_cyclic(X_train_f)
X_val_f   = add_wind_dir_cyclic(X_val_f)

# --- 3) winsorize precip on TRAIN, then log1p everywhere ---
precip_col = "precipitation_mm"
winsor_upper = 0.995  # 99.5th pct on train
cap_hi = np.nanpercentile(X_train_f[precip_col].astype(float), winsor_upper*100.0)
cap_lo = 0.0  # keep >= 0

def clip_and_log1p_precip(df):
    out = df.copy()
    out[precip_col] = out[precip_col].astype(float).clip(lower=cap_lo, upper=cap_hi)
    out["precipitation_mm_log1p"] = np.log1p(out[precip_col])
    return out

X_train_f = clip_and_log1p_precip(X_train_f)
X_val_f   = clip_and_log1p_precip(X_val_f)

# --- 4) standardize continuous *controls* on TRAIN; keep RAW demand for polynomial ---
to_std = [
    "wind_speed_mps",
    "temperature_celsius",
    "surface_net_solar_radiation_joules_per_m2",
    "precipitation_mm_log1p",
    "total_cloud_cover",
]
mu = {c: float(X_train_f[c].mean()) for c in to_std}
sd = {c: float(X_train_f[c].std(ddof=0) or 1.0) for c in to_std}  # avoid /0

def add_std(df):
    out = df.copy()
    for c in to_std:
        out[f"{c}_std"] = (out[c].astype(float) - mu[c]) / sd[c]
    return out

X_train_f = add_std(X_train_f)
X_val_f   = add_std(X_val_f)

# --- 5) polynomial in RAW demand q + 6) interactions with selected controls ---
q = "demand_met"

inter_features = [
    "wind_speed_mps_std",
    "temperature_celsius_std",
    "surface_net_solar_radiation_joules_per_m2_std",
    "wind_dir_sin",
    "wind_dir_cos",
]

lin_controls = inter_features + [
    "precipitation_mm_log1p_std",
    "total_cloud_cover_std",
    "hour_sin","hour_cos","dow_sin","dow_cos","doy_sin","doy_cos","is_weekend",
]

def build_design(df):
    out = df.copy()
    # poly in raw q
    out[f"{q}_p1"] = out[q].astype(float)
    out[f"{q}_p2"] = out[q].astype(float)**2
    out[f"{q}_p3"] = out[q].astype(float)**3
    out[f"{q}_p4"] = out[q].astype(float)**4
    # interactions q × feature
    for c in inter_features:
        if c in out.columns:
            out[f"{q}_x_{c}"] = out[q].astype(float) * out[c].astype(float)
    return out

D_train = build_design(X_train_f)
D_val   = build_design(X_val_f)

# columns to feed OLS (only those present)
poly_cols = [f"{q}_p1", f"{q}_p2", f"{q}_p3", f"{q}_p4"]
inter_cols = [c for c in [f"{q}_x_{c0}" for c0 in inter_features] if c in D_train.columns]
rhs_cols = [c for c in (poly_cols + inter_cols + lin_controls) if c in D_train.columns]

# --- 7) fit OLS with named design matrix ---
Xmat_train = sm.add_constant(D_train[rhs_cols], has_constant="add")
ols_model = sm.OLS(y_train.to_numpy(float), Xmat_train.to_numpy(float)).fit()
coef = pd.Series(ols_model.params, index=Xmat_train.columns)
print(ols_model.summary())

# --- helper: exact ME from coefficients ---
def me_from_df(df, coef_series):
    qv = df[q].astype(float).to_numpy()
    # pull safely (0 if absent)
    def c(name): return float(coef_series.get(name, 0.0))
    me = (
        c(f"{q}_p1")
        + 2.0*c(f"{q}_p2")*qv
        + 3.0*c(f"{q}_p3")*(qv**2)
        + 4.0*c(f"{q}_p4")*(qv**3)
    )
    for cfeat in inter_features:
        col = f"{q}_x_{cfeat}"
        if col in df.columns:
            me = me + c(col)*df[cfeat].astype(float).to_numpy()
    return me

# --- 8) predict ŷ and ME on each split ---
def predict_split(D, y_true=None, name="split"):
    Xm = sm.add_constant(D[rhs_cols], has_constant="add")
    y_pred = np.asarray(Xm.to_numpy(float) @ coef.values, dtype=float)
    me = me_from_df(D, coef)
    out = pd.DataFrame({
        "y_pred": y_pred,
        "ME": me,
    }, index=D.index)
    if y_true is not None:
        out["y_true"] = y_true.values
        print(
            f"{name.upper():5s} | r2={r2_score(out['y_true'], out['y_pred']):.3f} "
            f"rmse={root_mean_squared_error(out['y_true'], out['y_pred']):.1f} "
            f"mae={mean_absolute_error(out['y_true'], out['y_pred']):.1f} "
            f"n={len(out):,}"
        )
    return out

train_out = predict_split(D_train, y_train, name="train")
val_out   = predict_split(D_val,   y_val,   name="val")


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.482
Model:                            OLS   Adj. R-squared:                  0.482
Method:                 Least Squares   F-statistic:                 1.102e+06
Date:                Sun, 17 Aug 2025   Prob (F-statistic):               0.00
Time:                        12:25:58   Log-Likelihood:            -2.3783e+07
No. Observations:             2365200   AIC:                         4.757e+07
Df Residuals:                 2365197   BIC:                         4.757e+07
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.631e-15   1.47e-18   1110.902      0.0

In [170]:

# --- 1) time features (Fourier) + weekend ---
def add_time_features(df, ts_col="timestamp"):
    out = df.copy()
    t = pd.to_datetime(out[ts_col], errors="raise")
    # hour-of-day as fractional hour
    h = t.dt.hour + t.dt.minute/60.0
    out["hour_sin"] = np.sin(2*np.pi*h/24.0)
    out["hour_cos"] = np.cos(2*np.pi*h/24.0)
    # day-of-week: 0=Mon..6=Sun
    dow = t.dt.dayofweek.astype(float)
    out["dow_sin"] = np.sin(2*np.pi*dow/7.0)
    out["dow_cos"] = np.cos(2*np.pi*dow/7.0)
    # day-of-year
    doy = t.dt.dayofyear.astype(float)
    out["doy_sin"] = np.sin(2*np.pi*doy/365.0)
    out["doy_cos"] = np.cos(2*np.pi*doy/365.0)
    # weekend flag
    out["is_weekend"] = (t.dt.dayofweek >= 5).astype(int)
    return out

X_train_f = add_time_features(X_train)
X_val_f   = add_time_features(X_val)

# --- 2) wind direction cyclic encoding ---
def add_wind_dir_cyclic(df, col="wind_direction_meteorological"):
    out = df.copy()
    theta = np.deg2rad(out[col].astype(float))
    out["wind_dir_sin"] = np.sin(theta)
    out["wind_dir_cos"] = np.cos(theta)
    return out

X_train_f = add_wind_dir_cyclic(X_train_f)
X_val_f   = add_wind_dir_cyclic(X_val_f)

# --- 3) winsorize precip on TRAIN, then log1p everywhere ---
precip_col = "precipitation_mm"
winsor_upper = 0.995  # 99.5th pct on train
cap_hi = np.nanpercentile(X_train_f[precip_col].astype(float), winsor_upper*100.0)
cap_lo = 0.0  # keep >= 0

def clip_and_log1p_precip(df):
    out = df.copy()
    out[precip_col] = out[precip_col].astype(float).clip(lower=cap_lo, upper=cap_hi)
    out["precipitation_mm_log1p"] = np.log1p(out[precip_col])
    return out

X_train_f = clip_and_log1p_precip(X_train_f)
X_val_f   = clip_and_log1p_precip(X_val_f)

# --- 4) standardize continuous *controls* on TRAIN; keep RAW demand for polynomial ---
to_std = [
    "wind_speed_mps",
    "temperature_celsius",
    "surface_net_solar_radiation_joules_per_m2",
    "precipitation_mm_log1p",
    "total_cloud_cover",
]
mu = {c: float(X_train_f[c].mean()) for c in to_std}
sd = {c: float(X_train_f[c].std(ddof=0) or 1.0) for c in to_std}  # avoid /0

def add_std(df):
    out = df.copy()
    for c in to_std:
        out[f"{c}_std"] = (out[c].astype(float) - mu[c]) / sd[c]
    return out

X_train_f = add_std(X_train_f)
X_val_f   = add_std(X_val_f)

# --- 5) polynomial in RAW demand q + 6) interactions with selected controls ---
q = "demand_met"

# --- interactions: now without wind speed ---
inter_features = [
    "temperature_celsius_std",
    "surface_net_solar_radiation_joules_per_m2_std",
    "wind_dir_sin",
    "wind_dir_cos",
]

lin_controls = inter_features + [
    "precipitation_mm_log1p_std",
    "total_cloud_cover_std",
    "hour_sin","hour_cos","dow_sin","dow_cos","doy_sin","doy_cos","is_weekend",
]

# --- degree 3 polynomial in raw q ---
def build_design(df):
    out = df.copy()
    out[f"{q}_p1"] = out[q].astype(float)
    out[f"{q}_p2"] = out[q].astype(float)**2
    out[f"{q}_p3"] = out[q].astype(float)**3
    for c in inter_features:
        if c in out.columns:
            out[f"{q}_x_{c}"] = out[q].astype(float) * out[c].astype(float)
    return out

D_train = build_design(X_train_f)
D_val   = build_design(X_val_f)

poly_cols = [f"{q}_p1", f"{q}_p2", f"{q}_p3"]
inter_cols = [c for c in [f"{q}_x_{c0}" for c0 in inter_features] if c in D_train.columns]
rhs_cols = [c for c in (poly_cols + inter_cols + lin_controls) if c in D_train.columns]

# --- 7) fit OLS with named design matrix ---
Xmat_train = sm.add_constant(D_train[rhs_cols], has_constant="add")
ols_model = sm.OLS(y_train.to_numpy(float), Xmat_train.to_numpy(float)).fit()
coef = pd.Series(ols_model.params, index=Xmat_train.columns)
print(ols_model.summary())

# --- exact ME formula now stops at cubic ---
def me_from_df(df, coef_series):
    qv = df[q].astype(float).to_numpy()
    def c(name): return float(coef_series.get(name, 0.0))
    me = (
        c(f"{q}_p1")
        + 2.0*c(f"{q}_p2")*qv
        + 3.0*c(f"{q}_p3")*(qv**2)
    )
    for cfeat in inter_features:
        col = f"{q}_x_{cfeat}"
        if col in df.columns:
            me = me + c(col)*df[cfeat].astype(float).to_numpy()
    return me

# --- 8) predict ŷ and ME on each split ---
def predict_split(D, y_true=None, name="split"):
    Xm = sm.add_constant(D[rhs_cols], has_constant="add")
    y_pred = np.asarray(Xm.to_numpy(float) @ coef.values, dtype=float)
    me = me_from_df(D, coef)
    out = pd.DataFrame({
        "y_pred": y_pred,
        "ME": me,
    }, index=D.index)
    if y_true is not None:
        out["y_true"] = y_true.values
        print(
            f"{name.upper():5s} | r2={r2_score(out['y_true'], out['y_pred']):.3f} "
            f"rmse={root_mean_squared_error(out['y_true'], out['y_pred']):.1f} "
            f"mae={mean_absolute_error(out['y_true'], out['y_pred']):.1f} "
            f"n={len(out):,}"
        )
    return out

train_out = predict_split(D_train, y_train, name="train")
val_out   = predict_split(D_val,   y_val,   name="val")


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.569
Model:                            OLS   Adj. R-squared:                  0.569
Method:                 Least Squares   F-statistic:                 5.215e+05
Date:                Sun, 17 Aug 2025   Prob (F-statistic):               0.00
Time:                        12:26:01   Log-Likelihood:            -2.3565e+07
No. Observations:             2365200   AIC:                         4.713e+07
Df Residuals:                 2365193   BIC:                         4.713e+07
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       7.335e-06   1.85e-08    396.719      0.0

In [171]:
random_state = 12

In [172]:
preprocess = build_mef_preprocess_pipeline(timestamp_col="timestamp", use_winsorize_precip=True)

qspline = build_qspline_pipeline(
    preprocess,
    weather_linear_base=["wind_speed_mps_std","temperature_celsius_std"],
    n_splines_q=15, lam_q=10.0
)

qgam = build_qgam_pipeline(
    preprocess,
    smooth_cols=["surface_net_solar_radiation_joules_per_m2_std","wind_speed_mps_std","temperature_celsius_std"],
    linear_extra=["precipitation_mm_std","total_cloud_cover_std","wind_dir_sin","wind_dir_cos"],
    n_splines_q=15, lam_q=10.0, n_splines_other=12, lam_other=10.0
)

evaluate_mef_pipeline = evaluate_gam_pipeline

# Fit/eval each model separately
# Fit/eval each model separately
qspline.fit(pd.concat([X_train, y_train.rename("tons_co2")], axis=1), y_train)
qgam.fit(pd.concat(objs=[X_train, y_train.rename("tons_co2")], axis=1), y_train)

train_spline = evaluate_mef_pipeline(qspline, X_train, y_train)
train_gam    = evaluate_mef_pipeline(qgam,    X_train, y_train)
val_spline   = evaluate_mef_pipeline(qspline, X_val,   y_val)
val_gam      = evaluate_mef_pipeline(qgam,    X_val,   y_val)

print("TRAIN (spline):", metrics_from_frame(train_spline))
print("TRAIN (gam)   :", metrics_from_frame(train_gam))
print("VAL   (spline):", metrics_from_frame(val_spline))
print("VAL   (gam)   :", metrics_from_frame(val_gam))

TRAIN (spline): {'r2': 0.9007229178402355, 'rmse': 2466.808108215663, 'mae': 1895.805220643691, 'n_obs': 2365200}
TRAIN (gam)   : {'r2': 0.902350014367182, 'rmse': 2446.509784035914, 'mae': 1876.2662291433633, 'n_obs': 2365200}
VAL   (spline): {'r2': 0.7607804587011959, 'rmse': 2918.4935655969402, 'mae': 2355.146831214452, 'n_obs': 328320}
VAL   (gam)   : {'r2': 0.7635884458554831, 'rmse': 2901.314192780919, 'mae': 2349.7995390459955, 'n_obs': 328320}


In [173]:
preprocess = build_mef_preprocess_pipeline(timestamp_col="timestamp", use_winsorize_precip=True)

qspline = build_qspline_pipeline(
    preprocess,
    weather_linear_base=["total_cloud_cover_std"],
    n_splines_q=15, lam_q=10.0
)

qgam = build_qgam_pipeline(
    preprocess,
    smooth_cols=["surface_net_solar_radiation_joules_per_m2_std","wind_speed_mps_std","temperature_celsius_std", "precipitation_mm_std","wind_dir_sin","wind_dir_cos"],
    linear_extra=["total_cloud_cover_std",],
    n_splines_q=15, lam_q=10.0, n_splines_other=12, lam_other=10.0
)

evaluate_mef_pipeline = evaluate_gam_pipeline

# Fit/eval each model separately
qspline.fit(pd.concat([X_train, y_train.rename("tons_co2")], axis=1), y_train)
qgam.fit(pd.concat(objs=[X_train, y_train.rename("tons_co2")], axis=1), y_train)

train_spline = evaluate_mef_pipeline(qspline, X_train, y_train)
train_gam    = evaluate_mef_pipeline(qgam,    X_train, y_train)
val_spline   = evaluate_mef_pipeline(qspline, X_val,   y_val)
val_gam      = evaluate_mef_pipeline(qgam,    X_val,   y_val)

print("TRAIN (spline):", metrics_from_frame(train_spline))
print("TRAIN (gam)   :", metrics_from_frame(train_gam))
print("VAL   (spline):", metrics_from_frame(val_spline))
print("VAL   (gam)   :", metrics_from_frame(df=val_gam))


TRAIN (spline): {'r2': 0.8994856425956893, 'rmse': 2482.132238469184, 'mae': 1911.1260953291799, 'n_obs': 2365200}
TRAIN (gam)   : {'r2': 0.9026817135234814, 'rmse': 2442.3510761126913, 'mae': 1871.9748169092968, 'n_obs': 2365200}
VAL   (spline): {'r2': 0.7503071750569459, 'rmse': 2981.696564545364, 'mae': 2416.9335131209273, 'n_obs': 328320}
VAL   (gam)   : {'r2': 0.764982643506238, 'rmse': 2892.7465346089093, 'mae': 2341.276235681636, 'n_obs': 328320}


In [174]:
# --- QGAM v2: smooth {Q, wind, temp, solar, precip}; only cloud cover linear (+ wind_dir sin/cos linear)
def build_qgam_pipeline_v2(
    preprocess: Pipeline,
    fe_cols_prefixes=("month_", "hour_"),
    y_var="tons_co2",
    q_col="demand_met",
    q_std_col="demand_met_std",
    # smooth all meaningful nonlinear drivers
    smooth_cols=("wind_speed_mps_std",
                 "temperature_celsius_std",
                 "surface_net_solar_radiation_joules_per_m2_std",
                 "precipitation_mm_std"),
    # keep these linear
    linear_extra=("total_cloud_cover_std", "wind_dir_sin", "wind_dir_cos"),
    n_splines_q=16, lam_q=10.0,
    n_splines_other=20, lam_other=15.0,   # slightly stronger penalty to tame wiggles
    random_state=12,
):
    def _fe_names_after_fit(df_pre: pd.DataFrame) -> list[str]:
        return [c for c in df_pre.columns if c.startswith(fe_cols_prefixes)]

    est = QGAMRegressorPyGAM(
        y_var=y_var, q_col=q_col, q_std_col=q_std_col,
        smooth_cols=list(smooth_cols),
        weather_linear_cols=list(smooth_cols) + list(linear_extra),
        fe_cols=[],
        n_splines_q=n_splines_q, lam_q=lam_q,
        n_splines_other=n_splines_other, lam_other=lam_other,
        random_state=random_state,
    )

    class _AttachFENames(BaseEstimator, TransformerMixin):
        def __init__(self, estimator): self.estimator = estimator
        def fit(self, X, y=None):
            self.estimator.fe_cols = _fe_names_after_fit(X); return self
        def transform(self, X): return X

    return Pipeline([
        ("PreprocessMEF", preprocess),
        ("AttachFE", _AttachFENames(est)),
        ("QGAMRegressorPyGAM_v2", est),
    ])


In [175]:
# --- QSpline v2: Q spline + smooth {wind, temp, solar}; linear {precip, cloud, wind_dir}
def build_qspline_pipeline_v2(
    preprocess: Pipeline,
    fe_cols_prefixes=("month_", "hour_"),
    y_var="tons_co2",
    q_col="demand_met",
    q_std_col="demand_met_std",
    # smooth these weather terms (std-scaled from preprocess)
    smooth_cols=("wind_speed_mps_std",
                 "temperature_celsius_std",
                 "surface_net_solar_radiation_joules_per_m2_std"),
    # keep these linear
    linear_extra=("precipitation_mm_std", "total_cloud_cover_std", "wind_dir_sin", "wind_dir_cos"),
    n_splines_q=16, lam_q=10.0,         # a touch more flexibility for Q
    n_splines_other=20, lam_other=12.0, # more knots for temp/solar; modest penalty
    random_state=12,
):
    def _fe_names_after_fit(df_pre: pd.DataFrame) -> list[str]:
        return [c for c in df_pre.columns if c.startswith(fe_cols_prefixes)]

    # We use QGAMRegressorPyGAM under the hood so we can add a few smooth weather terms
    est = QGAMRegressorPyGAM(
        y_var=y_var, q_col=q_col, q_std_col=q_std_col,
        smooth_cols=list(smooth_cols),
        weather_linear_cols=list(smooth_cols) + list(linear_extra),
        fe_cols=[],  # filled at fit-time
        n_splines_q=n_splines_q, lam_q=lam_q,
        n_splines_other=n_splines_other, lam_other=lam_other,
        random_state=random_state,
    )

    class _AttachFENames(BaseEstimator, TransformerMixin):
        def __init__(self, estimator): self.estimator = estimator
        def fit(self, X, y=None):
            self.estimator.fe_cols = _fe_names_after_fit(X); return self
        def transform(self, X): return X

    return Pipeline([
        ("PreprocessMEF", preprocess),
        ("AttachFE", _AttachFENames(est)),
        ("QSplineRegressorPyGAM_v2", est),
    ])


In [176]:
preprocess = build_mef_preprocess_pipeline(timestamp_col="timestamp", use_winsorize_precip=True)

qspline_v2 = build_qspline_pipeline_v2(preprocess)
qgam_v2    = build_qgam_pipeline_v2(preprocess)

# Fit both on TRAIN (X must include the raw features; y is tons_co2)
qspline_v2.fit(pd.concat([X_train, y_train.rename("tons_co2")], axis=1), y_train)
qgam_v2.fit(pd.concat([X_train, y_train.rename("tons_co2")], axis=1), y_train)


evaluate_mef_pipeline = evaluate_gam_pipeline

train_spline = evaluate_mef_pipeline(qspline_v2, X_train, y_train)
train_gam    = evaluate_mef_pipeline(qgam_v2,    X_train, y_train)
val_spline   = evaluate_mef_pipeline(qspline_v2, X_val,   y_val)
val_gam      = evaluate_mef_pipeline(qgam_v2,    X_val,   y_val)

print("TRAIN (spline):", metrics_from_frame(train_spline))
print("TRAIN (gam)   :", metrics_from_frame(train_gam))
print("VAL   (spline):", metrics_from_frame(val_spline))
print("VAL   (gam)   :", metrics_from_frame(df=val_gam))


TRAIN (spline): {'r2': 0.9023497516729004, 'rmse': 2446.5130747875396, 'mae': 1876.2670131060654, 'n_obs': 2365200}
TRAIN (gam)   : {'r2': 0.9024198021173191, 'rmse': 2445.635401267366, 'mae': 1875.6468955007492, 'n_obs': 2365200}
VAL   (spline): {'r2': 0.7634820868875153, 'rmse': 2901.9667541276717, 'mae': 2350.0381272519685, 'n_obs': 328320}
VAL   (gam)   : {'r2': 0.7637625589516848, 'rmse': 2900.2456116722146, 'mae': 2347.757980548095, 'n_obs': 328320}


In [177]:
# 1) Preprocess with Fourier time + wind dir + winsor + log1p + standardize
preprocess = build_mef_preprocess_pipeline_2(timestamp_col="timestamp", use_winsorize_precip=True)

# 2) Build models using the *right* column names from preprocess_2
#    QSpline: smooth wind/temp/solar; linear: precip_log1p, cloud, wind_dir, time terms
qspline = build_qspline_pipeline_v2(
    preprocess,
    # smooth weather cols (std)
    smooth_cols=("wind_speed_mps_std",
                 "temperature_celsius_std",
                 "surface_net_solar_radiation_joules_per_m2_std"),
    # linear extras — NOTE precip *_log1p_std* here
    linear_extra=("precipitation_mm_log1p_std","total_cloud_cover_std","wind_dir_sin","wind_dir_cos",
                  "hour_sin","hour_cos","dow_sin","dow_cos","doy_sin","doy_cos","is_weekend"),
    n_splines_q=18, lam_q=10.0, n_splines_other=20, lam_other=12.0
)

#    QGAM: smooth wind/temp/solar/precip_log1p; linear: cloud, wind_dir, time
qgam = build_qgam_pipeline_v2(
    preprocess,
    smooth_cols=("wind_speed_mps_std",
                 "temperature_celsius_std",
                 "surface_net_solar_radiation_joules_per_m2_std",
                 "precipitation_mm_log1p_std"),
    linear_extra=("total_cloud_cover_std","wind_dir_sin","wind_dir_cos",
                  "hour_sin","hour_cos","dow_sin","dow_cos","doy_sin","doy_cos","is_weekend"),
    n_splines_q=18, lam_q=10.0, n_splines_other=20, lam_other=15.0
)

# 3) Fit (X must include tons_co2 for the final step; we concat y)
qspline.fit(pd.concat([X_train, y_train.rename("tons_co2")], axis=1), y_train)
qgam.fit(pd.concat([X_train, y_train.rename("tons_co2")], axis=1), y_train)

# 4) ALWAYS use the evaluator to get a DataFrame with y_true,y_pred,ME
evaluate_mef_pipeline = evaluate_gam_pipeline

train_spline = evaluate_mef_pipeline(qspline, X_train, y_train)
train_gam    = evaluate_mef_pipeline(qgam,    X_train, y_train)
val_spline   = evaluate_mef_pipeline(qspline, X_val,   y_val)
val_gam      = evaluate_mef_pipeline(qgam,    X_val,   y_val)

print("TRAIN (spline):", metrics_from_frame(train_spline))
print("TRAIN (gam)   :", metrics_from_frame(train_gam))
print("VAL   (spline):", metrics_from_frame(val_spline))
print("VAL   (gam)   :", metrics_from_frame(val_gam))


TRAIN (spline): {'r2': 0.8654680128898424, 'rmse': 2871.5980090999483, 'mae': 2251.7032523084335, 'n_obs': 2365200}
TRAIN (gam)   : {'r2': 0.8657315569102029, 'rmse': 2868.7839442176482, 'mae': 2249.0375048850465, 'n_obs': 2365200}
VAL   (spline): {'r2': 0.6673237518172881, 'rmse': 3441.686521386532, 'mae': 2868.835758898085, 'n_obs': 328320}
VAL   (gam)   : {'r2': 0.6693585909478375, 'rmse': 3431.144710326922, 'mae': 2859.9268315418162, 'n_obs': 328320}


In [178]:
def build_mef_preprocess_clean(
    timestamp_col: str = "timestamp",
    use_winsorize_precip: bool = True,
    winsor_upper: float = 0.995,
):
    """
    - Date/time cols + weekend
    - Fourier time (hour/week/doy)
    - Wind direction → sin/cos
    - (optional) winsorize precip, then log1p
    - Standardize continuous drivers (adds *_std)
    """
    steps = [
        ("DateTime", DateTimeFeatureAdder(timestamp_col=timestamp_col, drop_original=False)),
        ("Fourier",   TimeFourierAdder(timestamp_col=timestamp_col, add_hour=True, add_week=True, add_doy=True)),
        ("WindDir",   WindDirToCyclic(dir_col="wind_direction_meteorological",
                                      out_sin="wind_dir_sin", out_cos="wind_dir_cos", drop_original=True)),
    ]
    if use_winsorize_precip:
        steps.append(("WinsorizePrecip", Winsorize(columns=["precipitation_mm"], lower=0.0, upper=winsor_upper)))
    steps.append(("Log1pPrecip", Log1pColumns(columns=["precipitation_mm"])))

    cont_cols = [
        "demand_met",
        "wind_speed_mps",
        "temperature_celsius",
        "precipitation_mm_log1p",  # created by Log1pColumns
        "surface_net_solar_radiation_joules_per_m2",
        "total_cloud_cover",
    ]
    steps.append(("Standardize", StandardizeContinuous(columns=cont_cols, suffix="_std")))
    return Pipeline(steps, verbose=False)


In [179]:
def build_qspline_clean(
    preprocess: Pipeline,
    y_var="tons_co2",
    q_col="demand_met",
    q_std_col="demand_met_std",
    n_splines_q=18, lam_q=10.0, random_state=12,
):
    # linear regressors (already standardized) + time Fourier + weekend + wind dir
    linear_extras = [
        "wind_speed_mps_std","temperature_celsius_std",
        "precipitation_mm_log1p_std","surface_net_solar_radiation_joules_per_m2_std",
        "total_cloud_cover_std","wind_dir_sin","wind_dir_cos",
        "hour_sin","hour_cos","dow_sin","dow_cos","doy_sin","doy_cos","is_weekend",
    ]

    est = QSplineRegressorPyGAM(
        y_var=y_var, q_col=q_col, q_std_col=q_std_col,
        weather_linear_cols=linear_extras,
        fe_cols=[], n_splines_q=n_splines_q, lam_q=lam_q, random_state=random_state,
    )

    return Pipeline([
        ("PreprocessMEF", preprocess),
        ("QSplineRegressorPyGAM", est),
    ])


In [180]:
def build_qgam_clean(
    preprocess: Pipeline,
    y_var="tons_co2",
    q_col="demand_met",
    q_std_col="demand_met_std",
    smooth_cols=("wind_speed_mps_std",
                 "temperature_celsius_std",
                 "surface_net_solar_radiation_joules_per_m2_std",
                 "precipitation_mm_log1p_std"),
    linear_extra=("total_cloud_cover_std","wind_dir_sin","wind_dir_cos",
                  "hour_sin","hour_cos","dow_sin","dow_cos","doy_sin","doy_cos","is_weekend"),
    n_splines_q=18, lam_q=10.0,
    n_splines_other=20, lam_other=15.0,
    random_state=12,
):
    smooth_cols = list(smooth_cols)
    linear_extra = list(linear_extra)

    est = QGAMRegressorPyGAM(
        y_var=y_var, q_col=q_col, q_std_col=q_std_col,
        smooth_cols=smooth_cols,
        weather_linear_cols=smooth_cols + linear_extra,   # smooth subset is detected internally
        fe_cols=[],
        n_splines_q=n_splines_q, lam_q=lam_q,
        n_splines_other=n_splines_other, lam_other=lam_other,
        random_state=random_state,
    )

    return Pipeline([
        ("PreprocessMEF", preprocess),
        ("QGAMRegressorPyGAM", est),
    ])


In [181]:
preprocess = build_mef_preprocess_clean(timestamp_col="timestamp", use_winsorize_precip=True)

qspline = build_qspline_clean(preprocess)
qgam    = build_qgam_clean(preprocess)

# fit (concat y so final estimator can see y_var in the same frame)
qspline.fit(pd.concat([X_train, y_train.rename("tons_co2")], axis=1), y_train)
qgam.fit(pd.concat([X_train, y_train.rename("tons_co2")], axis=1), y_train)

# evaluate
evaluate_mef_pipeline = evaluate_gam_pipeline
train_spline = evaluate_mef_pipeline(qspline, X_train, y_train)
train_gam    = evaluate_mef_pipeline(qgam,    X_train, y_train)
val_spline   = evaluate_mef_pipeline(qspline, X_val,   y_val)
val_gam      = evaluate_mef_pipeline(qgam,    X_val,   y_val)

print("TRAIN (spline):", metrics_from_frame(train_spline))
print("TRAIN (gam)   :", metrics_from_frame(train_gam))
print("VAL   (spline):", metrics_from_frame(val_spline))
print("VAL   (gam)   :", metrics_from_frame(val_gam))


TRAIN (spline): {'r2': 0.8640675523558579, 'rmse': 2886.505793901197, 'mae': 2260.7960070990057, 'n_obs': 2365200}
TRAIN (gam)   : {'r2': 0.8657315569102029, 'rmse': 2868.783944217649, 'mae': 2249.037504885047, 'n_obs': 2365200}
VAL   (spline): {'r2': 0.6634287721344834, 'rmse': 3461.775556604186, 'mae': 2876.665516368778, 'n_obs': 328320}
VAL   (gam)   : {'r2': 0.6693585909478403, 'rmse': 3431.144710326907, 'mae': 2859.9268315417958, 'n_obs': 328320}


In [182]:
class PolynomialMERegressorRidge(BaseEstimator, TransformerMixin):
    """
    y = tons_co2
    Features:
      - Q_std, Q_std^2
      - drivers_std (linear)
      - (Q_std × driver_std) for each driver_std
      - time_cols (linear, e.g., hour_sin/cos, dow_sin/cos, doy_sin/cos, is_weekend)
    RidgeCV for stability. Provides y_pred and ME in ORIGINAL Q units.
    """
    def __init__(
        self,
        y_var="tons_co2",
        q_col="demand_met",
        q_std_col="demand_met_std",
        driver_std_cols=(
            "temperature_celsius_std",
            "wind_speed_mps_std",
            "surface_net_solar_radiation_joules_per_m2_std",
            "precipitation_mm_log1p_std",
            "total_cloud_cover_std",
            "wind_dir_sin",
            "wind_dir_cos",
        ),
        time_cols=("hour_sin","hour_cos","dow_sin","dow_cos","doy_sin","doy_cos","is_weekend"),
        alphas=(1e-3, 1e-2, 1e-1, 1, 3, 10, 30, 100),
        cv=5,
        fit_intercept=True,
        random_state=12,
    ):
        self.y_var = y_var
        self.q_col = q_col
        self.q_std_col = q_std_col
        self.driver_std_cols = list(driver_std_cols)
        self.time_cols = list(time_cols)
        self.alphas = tuple(alphas)
        self.cv = int(cv)
        self.fit_intercept = bool(fit_intercept)
        self.random_state = random_state

        # learned
        self.model_ = None
        self.q_std_train_ = None
        self.feature_names_ = None
        self.coef_series_ = None

    # ---------- design matrix ----------
    def _design(self, df: pd.DataFrame) -> np.ndarray:
        Qs = df[self.q_std_col].astype(float).to_numpy()
        X_blocks = []

        # Q terms
        X_blocks.append(Qs.reshape(-1,1))              # Q_std
        X_blocks.append((Qs**2).reshape(-1,1))         # Q_std^2

        # drivers (linear)
        Z = []
        for c in self.driver_std_cols:
            z = df[c].astype(float).to_numpy()
            Z.append(z.reshape(-1,1))
        Z = np.hstack(Z) if Z else np.empty((len(df),0))
        if Z.size:
            X_blocks.append(Z)

        # Q × driver interactions
        if Z.size:
            X_blocks.append(Z * Qs.reshape(-1,1))

        # time features (linear, already in [-1,1] except is_weekend)
        T = []
        for c in self.time_cols:
            if c in df.columns:
                T.append(df[c].astype(float).to_numpy().reshape(-1,1))
        T = np.hstack(T) if T else np.empty((len(df),0))
        if T.size:
            X_blocks.append(T)

        X = np.hstack(X_blocks) if X_blocks else np.empty((len(df),0))

        # names in the same order
        names = []
        names += ["Q_std","Q_std2"]
        names += list(self.driver_std_cols)
        names += [f"Q_std:{c}" for c in self.driver_std_cols]
        names += [c for c in self.time_cols if c in df.columns]

        self.feature_names_ = names
        return X

    # ---------- fit / predict ----------
    def fit(self, X: pd.DataFrame, y=None):
        if y is None:
            if self.y_var not in X.columns:
                raise KeyError(f"'{self.y_var}' must be present if y=None")
            y_arr = X[self.y_var].astype(float).to_numpy()
        else:
            y_arr = (y.iloc[:,0] if isinstance(y, pd.DataFrame) else y).astype(float).to_numpy()

        # need raw Q to compute std(Q) for chain rule
        if self.q_col not in X.columns or self.q_std_col not in X.columns:
            raise KeyError(f"Expect both '{self.q_col}' and '{self.q_std_col}' in X")

        self.q_std_train_ = float(X[self.q_col].astype(float).std(ddof=0)) or 1.0

        # design
        D = self._design(X)
        # model
        self.model_ = RidgeCV(alphas=self.alphas, cv=self.cv, fit_intercept=self.fit_intercept, scoring="r2")
        self.model_.fit(D, y_arr)

        # store coefficients with names for interpretation
        self.coef_series_ = pd.Series(self.model_.coef_, index=self.feature_names_)
        return self

    def predict(self, X: pd.DataFrame, predict_type: str = "ME"):
        if self.model_ is None:
            raise RuntimeError("Model not fitted")

        if predict_type.lower() == "y":
            D = self._design(X)
            return self.model_.predict(D)

        # ME: derivative wrt Q in ORIGINAL units
        # dY/dQ_std = β_Q + 2β_Q2*Q_std + Σ β_{Q:z} * z_std
        Qs = X[self.q_std_col].astype(float).to_numpy()
        beta = self.coef_series_
        # Safe getters (0 if missing)
        def b(name): return float(beta.get(name, 0.0))

        dY_dQstd = (
            b("Q_std")
            + 2.0 * b("Q_std2") * Qs
            + sum(b(f"Q_std:{c}") * X[c].astype(float).to_numpy()
                  for c in self.driver_std_cols if f"Q_std:{c}" in beta.index)
        )
        # chain rule to original Q
        return dY_dQstd * (1.0 / float(self.q_std_train_))

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        df = X.copy()
        df["y_pred"] = self.predict(df, predict_type="y")
        df["ME"] = self.predict(df, predict_type="ME")
        return df

    # handy exposers
    @property
    def coef_(self) -> pd.Series:
        return self.coef_series_.copy() if self.coef_series_ is not None else None

    @property
    def alpha_(self) -> float:
        return float(getattr(self.model_, "alpha_", np.nan))


In [183]:
# 1) Preprocess (already defined in your codebase)
preprocess = build_mef_preprocess_pipeline_2(timestamp_col="timestamp", use_winsorize_precip=True)

# 2) Final estimator: choose drivers present after preprocess_2
drivers = [
    "temperature_celsius_std",
    "wind_speed_mps_std",
    "surface_net_solar_radiation_joules_per_m2_std",
    "precipitation_mm_log1p_std",
    "total_cloud_cover_std",
    "wind_dir_sin",
    "wind_dir_cos",
]

time_cols = ["hour_sin","hour_cos","dow_sin","dow_cos","doy_sin","doy_cos","is_weekend"]

poly_ridge = PolynomialMERegressorRidge(
    y_var="tons_co2",
    q_col="demand_met",
    q_std_col="demand_met_std",
    driver_std_cols=drivers,
    time_cols=time_cols,
    alphas=(1e-3, 1e-2, 1e-1, 1, 3, 10, 30, 100),
    cv=5,
    fit_intercept=True,
    random_state=12,
)

from sklearn.pipeline import Pipeline
ridge_poly_pipeline = Pipeline([
    ("PreprocessMEF", preprocess),
    ("PolyRidgeME", poly_ridge),
])

# 3) Fit on TRAIN (concat y so final step can see y_var column)
ridge_poly_pipeline.fit(pd.concat([X_train, y_train.rename("tons_co2")], axis=1), y_train)

# 4) Evaluate with your existing helper (returns y_true, y_pred, ME)
evaluate_mef_pipeline = evaluate_gam_pipeline  # re-use your function

train_eval = evaluate_mef_pipeline(ridge_poly_pipeline, X_train, y_train)
val_eval   = evaluate_mef_pipeline(ridge_poly_pipeline, X_val,   y_val)

print("TRAIN:", metrics_from_frame(train_eval))
print("VAL  :", metrics_from_frame(val_eval))

# 5) Peek at coefficients and chosen alpha
est = ridge_poly_pipeline.named_steps["PolyRidgeME"]
print("Chosen alpha:", est.alpha_)
print(est.coef_.sort_values(key=np.abs, ascending=False).head(25))


TRAIN: {'r2': 0.8668213252728539, 'rmse': 2857.118211998734, 'mae': 2233.621682211388, 'n_obs': 2365200}
VAL  : {'r2': 0.6759170708920273, 'rmse': 3396.9448119661834, 'mae': 2802.7023490758033, 'n_obs': 328320}
Chosen alpha: 0.001
Q_std                                                  7871.937154
hour_cos                                               5555.744017
doy_cos                                                5065.526497
doy_sin                                                2803.229579
surface_net_solar_radiation_joules_per_m2_std          1206.847810
Q_std:surface_net_solar_radiation_joules_per_m2_std    -607.674321
hour_sin                                                570.722137
Q_std2                                                 -515.553924
temperature_celsius_std                                 474.836922
wind_speed_mps_std                                     -401.950392
Q_std:temperature_celsius_std                           320.432954
total_cloud_cover_std           